## Imports and constants

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import statsmodels.api as sm

import os
import time
import random
import itertools

from copy import copy
from cycler import cycler
from datetime import date
from scipy.optimize import curve_fit
from statannot import add_stat_annotation

pd.set_option('mode.chained_assignment',None)

In [2]:
import re
dateFormatExcel = re.compile('\d{2}/\d{2}/\d{4}')
dateFormatOk = re.compile('\d{2}-\d{2}-\d{2}')

In [3]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib widget 
# %matplotlib inline
matplotlib.rcParams.update({'figure.autolayout': True})


SMALLER_SIZE = 10
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=MEDIUM_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALLER_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

# prop_cycle = plt.rcParams['axes.prop_cycle']
# colors = prop_cycle.by_key()['color']
new_color_list = color=['#ff7f0e', '#1f77b4', '#2ca02c', '#d62728', '#9467bd', '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']
new_color_cycle = cycler(color=new_color_list)
plt.rcParams['axes.prop_cycle'] = new_color_cycle

In [4]:
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Range1d
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10
from bokeh.layouts import gridplot
output_notebook()

Loading BokehJS ...

In [10]:
mainDir = "C://Users//JosephVermeil//Desktop//ActinCortexAnalysis"
# mainDir = "C://Users//josep//Desktop//ActinCortexAnalysis"
experimentalDataDir = os.path.join(mainDir, "ExperimentalData")
dataDir = os.path.join(mainDir, "DataAnalysis")
figDir = os.path.join(dataDir, "Figures")
todayFigDir = os.path.join(figDir, "Historique//" + str(date.today()))
timeSeriesDataDir = os.path.join(dataDir, "TimeSeriesData")

In [11]:
# Display all detected time Series Data Files.

# allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
# allTimeSeriesDataFiles

Some generic useful subfunctions:

In [12]:
def get_R2(Y1, Y2):
    meanY = np.mean(Y1)
    meanYarray = meanY*np.ones(len(Y1))
    SST = np.sum((Y1-meanYarray)**2)
    SSE = np.sum((Y2-meanYarray)**2)
    R2 = SSE/SST
    return(R2)

def getDictAggMean(df):
    dictAggMean = {}
    for c in df.columns:
#         t = df[c].dtype
#         print(c, t)
        try :
            if np.array_equal(df[c], df[c].astype(bool)):
                dictAggMean[c] = 'min'
            else:
                try:
                    np.mean(df[c])
                    dictAggMean[c] = 'mean'
                except:
                    dictAggMean[c] = 'first'
        except:
                dictAggMean[c] = 'first'
    return(dictAggMean)

def findFirst(x, A):
    idx = (A==x).view(bool).argmax()
    return(idx)

def fitLine(X, Y):
    X = sm.add_constant(X)
    model = sm.OLS(Y, X)
    results = model.fit()
    params = results.params # Y=a*X+b ; params[0] = b,  params[1] = a
#     print(dir(results))
#     R2 = results.rsquared
#     ci = results.conf_int(alpha=0.05)
#     CovM = results.cov_params()
#     p = results.pvalues

# This is how are computed conf_int:
#
#     bse = results.bse
#     dist = stats.t
#     alpha = 0.05
#     q = dist.ppf(1 - alpha / 2, results.df_resid)
#     params = results.params
#     lower = params - q * bse
#     upper = params + q * bse
#     print(lower, upper)
    
    return(results.params, results)

def archiveFig(fig, ax, name='auto', figDir = todayFigDir, figSubDir=''):
    if not os.path.exists(figDir):
        os.makedirs(figDir)
    
    saveDir = os.path.join(todayFigDir, figSubDir)
    if not os.path.exists(saveDir):
        os.makedirs(saveDir)
    
    if name != 'auto':
        fig.savefig(os.path.join(saveDir, name + '.png'))
    
    else:
        suptitle = fig._suptitle.get_text()
        if len(suptitle) > 0:
            name = suptitle
            fig.savefig(os.path.join(saveDir, name + '.png'))
        
        else:
            try:
                N = len(ax)
                ax = ax[0]
            except:
                N = 1
                ax = ax
                
            xlabel = ax.get_xlabel()
            ylabel = ax.get_ylabel()
            if len(xlabel) > 0 and len(ylabel) > 0:
                name = ylabel + ' Vs ' + xlabel
                if N > 1:
                    name = name + '___etc'
                fig.savefig(os.path.join(saveDir, name + '.png'))
            
            else:
                title = ax.get_title()
                if len(title) > 0:
                    if N > 1:
                        name = name + '___etc'
                    fig.savefig(os.path.join(saveDir, name + '.png'))
                
                else:
                    figNum = gcf().number
                    name = 'figure ' + str(figNum) 
                    fig.savefig(os.path.join(saveDir, name + '.png'))

Note: How to compute confidence invetervals of fitted parameters with (1-alpha) confidence:

    0) from scipy import stats
    1) df = nb_pts - nb_parms ; se = diag(cov)**0.5
    2) Student t coefficient : q = stat.t.ppf(1 - alpha / 2, df)
    3) ConfInt = [params - q*se, params + q*se]

In [13]:
# Test of the fitLine function

# Npts = 10
# seed = 10
# std = 2

# np.random.seed(seed)
# X = np.array([i for i in range(Npts)])
# Y = np.array([i for i in range(Npts)])
# Y = Y + np.random.normal(0, std, Npts)
# p, R = fitLine(X, Y)

# Ypred = p[1]*X+p[0]
# fig, ax = plt.subplots(1,1)
# ax.plot(X, Y, 'bo')
# ax.plot(X, Ypred, 'r-', label='best fit')
# ax.legend(loc='best')
# fig.show()

In [14]:
# Test of the lmfit library // <examples/doc_model_gaussian.py>

# import matplotlib.pyplot as plt
# from lmfit import Model

# np.random.seed(seed)

# x = np.array([i for i in range(Npts)])
# y = np.array([i for i in range(Npts)])
# y = y + np.random.normal(0, std, Npts)


# # def gaussian(x, amp, cen, wid):
# #     """1-d gaussian: gaussian(x, amp, cen, wid)"""
# #     return (amp / (sqrt(2*pi) * wid)) * exp(-(x-cen)**2 / (2*wid**2))

# def linear(x, a, b):
#     """linear function"""
#     return (a*x + b)

# # gmodel = Model(gaussian)
# # result = gmodel.fit(y, x=x, amp=5, cen=5, wid=1)

# # print(result.fit_report())

# # plt.plot(x, y, 'bo')
# # plt.plot(x, result.init_fit, 'k--', label='initial fit')
# # plt.plot(x, result.best_fit, 'r-', label='best fit')
# # plt.legend(loc='best')
# # plt.show()

# lmodel = Model(linear)
# result = lmodel.fit(y, x=x, a=1.5, b=-1)

# print(result.fit_report())

# fig, ax = plt.subplots(1,1)
# ax.plot(x, y, 'bo')
# # ax.plot(x, result.init_fit, 'k--', label='initial fit')
# ax.plot(x, result.best_fit, 'r-', label='best fit')
# ax.legend(loc='best')
# fig.show()

# # print(dir(result))

In [15]:
# Test of the curve_fit function and subsequent computation of confidence intervals

# np.random.seed(seed)

# x = np.array([i for i in range(Npts)])
# y = np.array([i for i in range(Npts)])
# y = y + np.random.normal(0, std, Npts)

# def linear(x, a, b):
#     """linear function"""
#     return (a*x + b)


# initialParameters = [1.5, -1]
# #     print(initialParameters)

# # bounds on parameters - initial parameters must be within these
# lowerBounds = (-np.Inf, -np.Inf)
# upperBounds = (np.Inf, np.Inf)
# parameterBounds = [lowerBounds, upperBounds]

# # fittedParameters, pcov = curve_fit(chadwickModel, hCompr, fCompr, initialParameters, bounds = parameterBounds)
# fittedParameters, pcov = curve_fit(linear, x, y, initialParameters, bounds = parameterBounds)

# A, B = fittedParameters
# print(A, B)
# ypredict = linear(x, A, B)

# varA = pcov[0,0]
# print(pcov)
# print(pcov[0,0]**0.5)
# SSR = np.sum((np.array(y)-np.array(ypredict))**2)
# seA = ((SSR/(len(x)-2))*varA)**0.5
# seA = (varA)**0.5
# confIntA = st.t.interval(alpha=0.95, df=len(x)-2, loc=A, scale=seA)
# confIntAHalfWidthBis = st.t.ppf(0.975, len(x)-2) * seA
# print(confIntAHalfWidthBis)
# # confIntA = (A + seA*st.t.interval(alpha=0.95, df=len(x), loc=0, scale=1)[0], A + seA*st.t.interval(alpha=0.95, df=len(x), loc=0, scale=1)[1])
# confIntAWidth = confIntA[1] - confIntA[0]
# confIntAHalfWidth = confIntAWidth/2
# R2 = get_R2(y,ypredict)

# print(str(A) + ' +/- ' + str(confIntAHalfWidth))

# fig, ax = plt.subplots(1,1)
# ax.plot(x, y, 'bo')
# ax.plot(x, ypredict, 'r-', label='best fit')
# ax.legend(loc='best')
# fig.show()

In [16]:
# Test of the get_R2 function.
# T = df['T']
# Y = df['D3']
# plt.plot(T,Y)
# p, residuals, rank, singular_values, rcond = np.polyfit(T, Y, deg=5, full=True)
# plt.plot(T, Y)
# Y2 = np.zeros(len(T))
# for i in range(len(T)):
#     deg = len(p)-1
#     for k in range(deg+1):
#         Y2[i] += p[k]*(T[i]**(deg-k))
# plt.plot(T,Y2)
# get_R2(Y, Y2)

## TimeSeries functions

In [111]:
def getCellTimeSeriesData(cellID):
    allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv"))]
    fileFound = False
    nFile = len(allTimeSeriesDataFiles)
    iFile = 0
    while (not fileFound) and (iFile < nFile):
        f = allTimeSeriesDataFiles[iFile]
        if f.startswith(cellID):
            timeSeriesDataFilePath = os.path.join(timeSeriesDataDir, f)
            timeSeriesDataFrame = pd.read_csv(timeSeriesDataFilePath, sep=';')
            fileFound = True
        iFile += 1
    if not fileFound:
        timeSeriesDataFrame = pd.DataFrame([])
    else:
        for c in timeSeriesDataFrame.columns:
                if 'Unnamed' in c:
                    timeSeriesDataFrame = timeSeriesDataFrame.drop([c], axis=1)
    return(timeSeriesDataFrame)

def plotCellTimeSeriesData(cellID):
    X = 'T'
    Y = np.array(['B', 'F', 'dx', 'dy', 'dz', 'D2', 'D3'])
    units = np.array([' (mT)', ' (pN)', ' (µm)', ' (µm)', ' (µm)', ' (µm)', ' (µm)'])
    timeSeriesDataFrame = getCellTimeSeriesData(cellID)
    if not timeSeriesDataFrame.size == 0:
#         plt.tight_layout()
#         fig.show() # figsize=(20,20)
        axes = timeSeriesDataFrame.plot(x=X, y=Y, kind='line', ax=None, subplots=True, sharex=True, sharey=False, layout=None, \
                       figsize=(8,10), use_index=True, title = cellID + '- Time dependant data', grid=None, legend=False, style=None, logx=False, logy=False, \
                       loglog=False, xticks=None, yticks=None, xlim=None, ylim=None, rot=None, fontsize=None, colormap=None, \
                       table=False, yerr=None, xerr=None, secondary_y=False, sort_columns=False)
        plt.gcf().tight_layout()
        for i in range(len(Y)):
            axes[i].set_ylabel(Y[i] + units[i])
        
    else:
        print('cell not found')
        
def addExcludedCell(cellID, motive):
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'r')
    lines = f.readlines()
    nLines = len(lines)
    excludedCellsList = []
    for iLine in range(nLines):
        line = lines[iLine]
        splitLine = line[:-1].split(',')
        excludedCellsList.append(splitLine[0])
    if cellID in excludedCellsList:
        newlines = copy(lines)
        iLineOfInterest = excludedCellsList.index(cellID)
        if motive not in newlines[iLineOfInterest][:-1].split(','):
            newlines[iLineOfInterest] = newlines[iLineOfInterest][:-1] + ',' + motive + '\n'            
    else:
        newlines = copy(lines)
        newlines.append('' + cellID + ',' + motive + '\n')
    f.close()
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'w')
    f.writelines(newlines)
    
def getExcludedCells():
    f = open(os.path.join(experimentalDataDir, 'ExcludedCells.txt'), 'r')
    lines = f.readlines()
    nLines = len(lines)
    excludedCellsDict = {}
    for iLine in range(nLines):
        line = lines[iLine]
        splitLine = line[:-1].split(',')
        excludedCellsDict[splitLine[0]] = splitLine[1:]
    return(excludedCellsDict)

In [113]:
df = getCellTimeSeriesData('21-01-18_M1_P1_C1')
# plotCellTimeSeriesData('21-01-21_M1_P1_C11')
# plotCellTimeSeriesData('21-04-21_M1_P1_C5')
df

,idxCompression,T,Tabs,B,F,dx,dy,dz,D2,D3
0,0,0.726,16717.461,6.15123,142.903483,4.598671,0.425696,0.278339,4.618332,4.626712
1,0,1.326,16718.061,6.15369,142.810633,4.600063,0.434620,0.262935,4.620549,4.628025
2,0,1.926,16718.661,6.15492,143.290588,4.597342,0.440633,0.246409,4.618410,4.624979
3,0,2.526,16719.261,6.15369,143.970811,4.594241,0.436266,0.228916,4.614908,4.620582
4,0,3.125,16719.860,6.15492,143.961890,4.595886,0.437468,0.210633,4.616660,4.621462
...,...,...,...,...,...,...,...,...,...,...
1590,0,119.675,16836.410,6.15123,149.158059,4.568418,0.416835,-0.050157,4.587395,4.587669
1591,0,120.275,16837.010,6.15123,148.984697,4.568861,0.417595,-0.078146,4.587905,4.588571
1592,0,120.875,16837.610,6.15369,148.166412,4.573797,0.420506,-0.107096,4.593087,4.594335
1593,0,121.474,16838.209,6.15492,148.665066,4.568797,0.423038,-0.136590,4.588341,4.590373


## GlobalTables functions

### Experimental conditions

In [19]:
def getExperimentalConditions(save = False):
    # Getting the table
    experimentalDataFile = 'ExperimentalConditions.csv'
    experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
    expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')
    
    # Cleaning the table
    try:
        for c in expConditionsDF.columns:
            if 'Unnamed' in c:
                expConditionsDF = expConditionsDF.drop([c], axis=1)
        expConditionsDF = expConditionsDF.convert_dtypes()

        listTextColumns = []
        for col in expConditionsDF.columns:
            try:
                if expConditionsDF[col].dtype == 'string':
                    listTextColumns.append(col)
            except:
                aaaa=0
                #Ok

        expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

        expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
        try:
            expConditionsDF['optical index correction'] = \
                      expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
                    / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
        except:
            print('optical index correction already in ' + str(expConditionsDF['optical index correction'].dtype) + ' type.')

        expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
        expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

        try:
            expConditionsDF['ramp field'] = \
            expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])
        except:
            aaaa=0
            #Ok

        dateExemple = expConditionsDF.loc[expConditionsDF.index[1],'date']

        if re.match(dateFormatExcel, dateExemple):
            print('dates corrected')
            expConditionsDF.loc[1:,'date'] = expConditionsDF.loc[1:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])        
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'ExperimentalConditions.csv'
        savePath = os.path.join(experimentalDataDir, saveName)
        expConditionsDF.to_csv(savePath, sep=';')

    expConditionsDF['manipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']
    
    return(expConditionsDF)

In [20]:
getExperimentalConditions(save=False).head()

Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.


,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,cell subtype,bead type,bead diameter,normal field,ramp field,compression duration,with fluo images,bacteria,comments,manipID
0,DEFAULT,DEFAULT,DEFAULT,DEFAULT,DEFAULT,100X,15.8,oil,0.875,1.15,...,DEFAULT,M450,4503,5,['3'. '40'],1s,False,<NA>,<NA>,DEFAULT_DEFAULT
1,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,aSFL,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,20-08-04_M1
2,20-08-04,M2,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,aSFL,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,20-08-04_M2
3,20-08-05,M1,compressions and constant field,doxycyclin,BSA coated glass,100X,15.8,oil,0.875,1.15,...,aSFL,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,20-08-05_M1
4,20-08-05,M2,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,aSFL,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,20-08-05_M2


### Constant Field

In [21]:
listColumnsCtField = ['date','cellName','cellID','manipID',\
                      'duration','medianRawB','medianThickness',\
                      '1stDThickness','9thDThickness','fluctuAmpli',\
                      'R2_polyFit','validated']

def analyseTimeSeries_ctField(tsDf):
    results = {}
    results['duration'] = np.max(tsDf['T'])
    results['medianRawB'] = np.median(tsDf.B)
    results['medianThickness'] = np.median(tsDf.D3)
    results['1stDThickness'] = np.percentile(tsDf.D3, 10)
    results['9thDThickness'] = np.percentile(tsDf.D3, 90)
    results['fluctuAmpli'] = results['9thDThickness'] - results['1stDThickness']
    results['validated'] = (results['1stDThickness'] > 0)
    X, Y = tsDf['T'], tsDf['D3']
    p, residuals, rank, singular_values, rcond = np.polyfit(X, Y, deg=5, full=True)
    Y2 = np.zeros(len(X))
    for i in range(len(X)):
        deg = len(p)-1
        for k in range(deg+1):
            Y2[i] += p[k]*(X[i]**(deg-k))
    results['R2_polyFit'] = get_R2(Y, Y2)
    return(results)



def createDataDict_ctField(list_ctFieldFiles):
    tableDict = {}
    tableDict['date'], tableDict['cellName'], tableDict['cellID'], tableDict['manipID'] = [], [], [], []
    tableDict['duration'], tableDict['medianRawB'], tableDict['medianThickness'] = [], [], []
    tableDict['1stDThickness'], tableDict['9thDThickness'], tableDict['fluctuAmpli'] = [], [], []
    tableDict['R2_polyFit'], tableDict['validated'] = [], []
    for f in list_ctFieldFiles:
        split_f = f.split('_')
        tableDict['date'].append(split_f[0])
        tableDict['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        tableDict['manipID'].append(split_f[0] + '_' + split_f[1])
        tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
        current_tsDf = pd.read_csv(tS_DataFilePath, ';')
        current_resultDict = analyseTimeSeries_ctField(current_tsDf)
        for k in current_resultDict.keys():
            tableDict[k].append(current_resultDict[k])
    return(tableDict)



def computeGlobalTable_ctField(task = 'fromScratch', fileName = 'Global_CtFieldData', save = False):
    ctFieldTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) \
                                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                                      and ('thickness' in f))]
#     print(ctFieldTimeSeriesDataFiles)
    if task == 'fromScratch':
        # create a dict containing the data
        tableDict = createDataDict_ctField(ctFieldTimeSeriesDataFiles) # MAIN SUBFUNCTION
        # create the table
        CtField_DF = pd.DataFrame(tableDict)
        
    elif task == 'updateExisting':
        # get existing table
        try:
            savePath = os.path.join(dataDir, (fileName + '.csv'))
            existing_CtField_DF = pd.read_csv(savePath, sep=';')
            for c in existing_CtField_DF.columns:
                if 'Unnamed' in c:
                    existing_CtField_DF = existing_CtField_DF.drop([c], axis=1)
        except:
            print('No existing table found')
        # find which of the time series files are new
        new_ctFieldTimeSeriesDataFiles = []
        for f in ctFieldTimeSeriesDataFiles:
            split_f = f.split('_')
            currentCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
            if currentCellID not in existing_CtField_DF.cellID.values:
                new_ctFieldTimeSeriesDataFiles.append(f)
        new_tableDict = createDataDict_ctField(new_ctFieldTimeSeriesDataFiles) # MAIN SUBFUNCTION
        # create the table with new data
        new_CtField_DF = pd.DataFrame(new_tableDict)
        # fuse the two
        new_CtField_DF.index += existing_CtField_DF.shape[0]
        CtField_DF = pd.concat([existing_CtField_DF, new_CtField_DF])
    
    dateExemple = CtField_DF.loc[CtField_DF.index[0],'date']
    if re.match(dateFormatExcel, dateExemple):
        CtField_DF.loc[:,'date'] = CtField_DF.loc[:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])
    
    if save:
        saveName = fileName + '.csv'
        savePath = os.path.join(dataDir, saveName)
        CtField_DF.to_csv(savePath, sep=';')
        
    return(CtField_DF)



def getGlobalTable_ctField(fileName = 'Global_CtFieldData'):
    try:
        savePath = os.path.join(dataDir, (fileName + '.csv'))
        CtField_DF = pd.read_csv(savePath, sep=';')
        for c in CtField_DF.columns:
            if 'Unnamed' in c:
                CtField_DF = CtField_DF.drop([c], axis=1)
        print('Extracted a table with ' + str(CtField_DF.shape[0]) + ' lines and ' + str(CtField_DF.shape[1]) + ' columns.')
        
    except:
        print('No existing table found')
        
    dateExemple = CtField_DF.loc[CtField_DF.index[0],'date']
    if re.match(dateFormatExcel, dateExemple):
        print('dates corrected')
        CtField_DF.loc[:,'date'] = CtField_DF.loc[:,'date'].apply(lambda x: x.split('/')[0] + '-' + x.split('/')[1] + '-' + x.split('/')[2][2:])
#         mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
    return(CtField_DF)

In [22]:
computeGlobalTable_ctField(task='updateExisting',save=False).head()

,date,cellName,cellID,manipID,duration,medianRawB,medianThickness,1stDThickness,9thDThickness,fluctuAmpli,R2_polyFit,validated
0,21-02-10,M1_P1_C1,21-02-10_M1_P1_C1,21-02-10_M1,393.550,6.0036,183.232388,103.105084,276.671473,173.566389,0.222994,1.0
1,21-02-10,M1_P1_C2,21-02-10_M1_P1_C2,21-02-10_M1,326.388,6.0036,-58.229968,-147.203982,41.303068,188.507051,0.228033,0.0
2,21-02-10,M1_P1_C3,21-02-10_M1_P1_C3,21-02-10_M1,327.997,6.0036,204.963636,123.738478,277.782516,154.044038,0.402168,1.0
3,21-02-10,M1_P1_C4,21-02-10_M1_P1_C4,21-02-10_M1,326.929,6.0036,203.321336,165.769700,282.439284,116.669584,0.261129,1.0
4,21-02-10,M1_P1_C5,21-02-10_M1_P1_C5,21-02-10_M1,260.260,6.0036,231.480125,169.220846,301.601770,132.380924,0.248455,1.0


In [23]:
getGlobalTable_ctField().head()

Extracted a table with 40 lines and 12 columns.


,date,cellName,cellID,manipID,duration,medianRawB,medianThickness,1stDThickness,9thDThickness,fluctuAmpli,R2_polyFit,validated
0,21-02-10,M1_P1_C1,21-02-10_M1_P1_C1,21-02-10_M1,393.550,6.0036,183.232388,103.105084,276.671473,173.566389,0.222994,1.0
1,21-02-10,M1_P1_C2,21-02-10_M1_P1_C2,21-02-10_M1,326.388,6.0036,-58.229968,-147.203982,41.303068,188.507051,0.228033,0.0
2,21-02-10,M1_P1_C3,21-02-10_M1_P1_C3,21-02-10_M1,327.997,6.0036,204.963636,123.738478,277.782516,154.044038,0.402168,1.0
3,21-02-10,M1_P1_C4,21-02-10_M1_P1_C4,21-02-10_M1,326.929,6.0036,203.321336,165.769700,282.439284,116.669584,0.261129,1.0
4,21-02-10,M1_P1_C5,21-02-10_M1_P1_C5,21-02-10_M1,260.260,6.0036,231.480125,169.220846,301.601770,132.380924,0.248455,1.0


### Mechanics

Workflow
* analyseTimeSeries_meca() analyse 1 file and return the dict (with the results of the analysis)
* createMecaDataDict() call the previous function on the given list of files and concatenate the results
* computeGlobalTable_meca() call the previous function and convert the dict to a DataFrame

In [120]:
listColumnsMeca = ['date','cellName','cellID','manipID',\
                   'compNum','compDuration','compStartTime','compAbsStartTime','compStartTimeThisDay',\
                   'initialThickness','minThickness','maxIndent','previousThickness','surroundingThickness',\
                   'validatedThickness',\
                   'ctFieldThickness','ctFieldFluctuAmpli',\
                   'H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth',\
                   'hysteresis',\
                   'critFit', 'validatedFit','comments'] # 'fitParams',

def compressionFitChadwick(hCompr, fCompr, DIAMETER):
    
    error = False
    
    def chadwickModel(h, E, H0):
        R = DIAMETER/2
        f = (np.pi*E*R*((H0-h)**2))/(3*H0)
        return(f)

    def inversedChadwickModel(f, E, H0):
        R = DIAMETER/2
        h = H0 - ((3*H0*f)/(np.pi*E*R))**0.5
        return(h)

    # some initial parameter values - must be within bounds
    initH0 = max(hCompr) # H0 ~ h_max
    initE = (3*max(hCompr)*max(fCompr))/(np.pi*(DIAMETER/2)*(max(hCompr)-min(hCompr))**2) # E ~ 3*H0*F_max / pi*R*(H0-h_min)²
#     initH0, initE = initH0*(initH0>0), initE*(initE>0)
    
    initialParameters = [initE, initH0]
#     print(initialParameters)

    # bounds on parameters - initial parameters must be within these
    lowerBounds = (0, 0)
    upperBounds = (np.Inf, np.Inf)
    parameterBounds = [lowerBounds, upperBounds]
    
#     testH = inversedChadwickModel(fCompr, initE, initH0)
#     fig, ax = plt.subplots(1,1)
#     ax.plot(hCompr,fCompr,'b-', linewidth = 0.8)
#     ax.plot(testH,fCompr,'kx', linewidth = 0.8)
#     ax.set_xlabel('h (nm)')
#     ax.set_ylabel('f (pN)')
#     fig.show()
#     print(initialParameters)

    try:
        params, covM = curve_fit(inversedChadwickModel, fCompr, hCompr, initialParameters, bounds = parameterBounds)

        # Previously I fitted with y=F and x=H, but it didn't work so well cause H(t) isn't monotonous:
        # params, covM = curve_fit(chadwickModel, hCompr, fCompr, initialParameters, bounds = parameterBounds)
        # Fitting with the 'inverse Chadwick model', with y=H and x=F is more convenient

        E, H0 = params
        hPredict = inversedChadwickModel(fCompr, E, H0)

        SSR = np.sum((hCompr-hPredict)**2)
        alpha = 0.975
        df = len(fCompr)-len(params)
        q = st.t.ppf(alpha, df) # Student coefficient
        R2 = get_R2(hCompr,hPredict)

        varE = covM[0,0]
        seE = (varE)**0.5
        E, seE = E*1e6, seE*1e6
        confIntE = [E-q*seE, E+q*seE]
        confIntEWidth = 2*q*seE

        varH0 = covM[1,1]
        seH0 = (varH0)**0.5
        confIntH0 = [H0-q*seH0, H0+q*seH0]
        confIntH0Width = 2*q*seH0
        
        
    except:
        error = True
        E, H0, hPredict, R2, confIntE, confIntH0 = -1, -1, np.ones(len(hCompr))*(-1), -1, [-1,-1], [-1,-1]
    
    return(E, H0, hPredict, R2, confIntE, confIntH0, error)



def analyseTimeSeries_meca(f, tsDF, expDf, listColumnsMeca, PLOT, PLOT_SHOW):
    
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]
    DIAMETER = thisExpDf.at[thisExpDf.index.values[0], 'bead diameter']
    
    results = {}
    for c in listColumnsMeca:
        results[c] = []
        
    Ncomp = max(tsDF['idxCompression'])
    NimgComp = np.sum((tsDF['idxCompression'] != 0))/Ncomp
    NimgCompTh = round(0.49999999999 + np.sum((tsDF['idxCompression'] != 0))/Ncomp)
    NimgBtwComp = np.sum((tsDF['idxCompression'] == 0))/Ncomp
    NimgBtwCompTh = round(0.4999999999 + np.sum((tsDF['idxCompression'] == 0))/Ncomp)
#     print('Ncomp : ' + str(Ncomp) + ' ; ' + 'NimgComp : ' + str(NimgComp) + '/' + str(NimgCompTh) + ' ; ' + 'NimgBtwComp : ' + str(NimgBtwComp) + '/' + str(NimgBtwCompTh))
#     if not NimgBtwComp%2 == 0:
#         print('Bug with the compressions sequence delimitation')


    # These values are computed once for the whole cell D3 time series, but since the table has 1 line per compression, 
    # that same value will be put in the table for each line corresponding to that cell
    ctFieldH = (tsDF.loc[tsDF['idxCompression'] == 0, 'D3'].values - DIAMETER)
    ctFieldThickness   = np.median(ctFieldH)
    ctFieldFluctuAmpli = np.percentile(ctFieldH,90) - np.percentile(ctFieldH,10)
    
    if PLOT:
        nColsSubplot = 5
        nRowsSubplot = ((Ncomp-1) // nColsSubplot) + 1
        fig1, ax1 = plt.subplots(1,1,figsize=(tsDF.shape[0]*(1/100),3))
#         fig2, ax2 = plt.subplots(1,Ncomp,figsize=(3*Ncomp,3))
        fig2, ax2 = plt.subplots(nRowsSubplot,nColsSubplot,figsize=(3*nColsSubplot,3*nRowsSubplot))
        ax1.plot(tsDF['T'].values, tsDF['D3'].values-DIAMETER, 'b-', linewidth = 0.75)

    
    for i in range(1, Ncomp+1):#Ncomp+1):

        # (1) Identifiers
        results['date'].append(split_f[0])
        results['cellName'].append(split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['cellID'].append(split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3])
        results['manipID'].append(split_f[0] + '_' + split_f[1])
        
        # (2) Segment the compression n°i
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i,:]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart+thisCompDf.shape[0]
        
        # Easy-to-get parameters
        results['compNum'].append(i)
        results['compDuration'].append(thisExpDf.at[thisExpDf.index.values[0], 'compression duration'])
        results['compStartTime'].append(thisCompDf['T'].values[0])
        results['compAbsStartTime'].append(thisCompDf['Tabs'].values[0])
        results['compStartTimeThisDay'].append(thisCompDf['Tabs'].values[0]) # Will be modifyed later !
        
        # (3) Inside the compression n°i, delimit the compression and relaxation phases
        
        # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
        # and the end of its decrease (typically when it goes back from 3 to 5)
        listB = thisCompDf.B.values
        offsetStart, offsetStop = 0, 0
        minB, maxB = min(listB), max(listB)
        thresholdB = (maxB-minB)/50
        
        k = 0
        while (listB[k] > minB+thresholdB) or (listB[-1-k] > minB+thresholdB):
            offsetStart += int(listB[k] > minB+thresholdB)
            offsetStop += int(listB[-1-k] > minB+thresholdB)
            k += 1
        jStart = offsetStart # Beginning of compression
        jMax = np.argmax(thisCompDf.B) # End of compression, beginning of relaxation
        jStop = thisCompDf.shape[0] - offsetStop # End of relaxation
        
        # Four arrays
        hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
        hRelax = (thisCompDf.D3.values[jMax+1:jStop] - DIAMETER)
        fCompr = (thisCompDf.F.values[jStart:jMax+1])
        fRelax = (thisCompDf.F.values[jMax+1:jStop])
        
        # Refinement of the compression delimitation.
        # Remove the 1-2 points at the begining where there is just the viscous relaxation of the cortex
        # because of the initial decrease of B and the cortex thickness increases.
        offsetStart2 = 0
        k = 0
        while (hCompr[k] < np.max(hCompr[k+1:min(k+10, len(hCompr))])) and k<len(hCompr)-10:
            offsetStart2 += 1
            k += 1
        # Better compressions arrays
        hCompr = hCompr[offsetStart2:]
        fCompr = fCompr[offsetStart2:]
        
        # Get the points of constant field preceding and surrounding the current compression
        # Ex : if the labview code was set so that there is 6 points of ct field before and after each compression,
        # previousPoints will contains D3[iStart-12:iStart]
        # surroundingPoints will contains D3[iStart-6:iStart] and D3[iStop:iStop+6]
        previousPoints = (tsDF.D3.values[max(0,iStart-(NimgBtwCompTh)):iStart]) - DIAMETER
        surroundingPoints = np.concatenate([tsDF.D3.values[max(0,iStart-(NimgBtwCompTh//2)):iStart],tsDF.D3.values[iStop:iStop+(NimgBtwCompTh//2)]]) - DIAMETER
        
        # Parameters relative to the thickness ( = D3-DIAMETER)
        results['initialThickness'].append(np.mean(hCompr[0:3]))
        results['minThickness'].append(np.min(hCompr))
        results['maxIndent'].append(results['initialThickness'][-1] - results['minThickness'][-1])
        results['previousThickness'].append(np.median(previousPoints))
        results['surroundingThickness'].append(np.median(surroundingPoints))
        results['ctFieldThickness'].append(ctFieldThickness)
        results['ctFieldFluctuAmpli'].append(ctFieldFluctuAmpli)
        
        validatedThickness = np.min([results['initialThickness'],results['minThickness'],results['previousThickness'],\
                                    results['surroundingThickness'],results['ctFieldThickness']]) > 0
        results['validatedThickness'].append(validatedThickness)

        # (4) Fit with Chadwick model of the force-thickness curve
        
        E, H0, hPredict, R2, confIntE, confIntH0, fitError = compressionFitChadwick(hCompr, fCompr, DIAMETER) # IMPORTANT SUBFUNCTION
        
        R2CRITERION = 0.9
        critFit = 'R2 > ' + str(R2CRITERION)
        results['critFit'].append(critFit)
        validatedFit = (R2 > R2CRITERION)
        
        if PLOT:
            # fig1
            if not fitError:
                if validatedFit:
                    ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'chartreuse', linestyle = '-', linewidth = 1)
                else:
                    ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'gold', linestyle = '-', linewidth = 1)
            else:
                ax1.plot(thisCompDf['T'].values, thisCompDf['D3'].values-DIAMETER, color = 'crimson', linestyle = '-', linewidth = 1)
            ax1.set_xlabel('t (s)')
            ax1.set_ylabel('h (nm)')
            fig1.suptitle(results['cellID'][-1])
            
            # fig2
            colSp = (i-1) % nColsSubplot
            rowSp = (i-1) // nColsSubplot
            # ax2[i-1] with the 1 line plot
            if nRowsSubplot == 1:
                thisAx2 = ax2[colSp]
            elif nRowsSubplot >= 1:
                thisAx2 = ax2[rowSp,colSp]
            thisAx2.plot(hCompr,fCompr,'b-', linewidth = 0.8)
            thisAx2.plot(hRelax,fRelax,'r-', linewidth = 0.8)
            titleText = results['cellID'][-1] + '__c' + str(i)
            legendText = ''
            thisAx2.set_xlabel('h (nm)')
            thisAx2.set_ylabel('f (pN)')
            if not fitError:
                legendText += 'H0 = {:.1f}nm\nE = {:.2e}Pa\nR2 = {:.3f}'.format(H0, E, R2)
                thisAx2.plot(hPredict,fCompr,'k--', linewidth = 0.8, label = legendText)
                thisAx2.legend(loc = 'upper right', prop={'size': 6})
                if not validatedFit:
                    titleText += '\nNON VALIDATED'
            else:
                titleText += '\nFIT ERROR'
            thisAx2.title.set_text(titleText)
            for item in ([thisAx2.title, thisAx2.xaxis.label, \
                          thisAx2.yaxis.label] + thisAx2.get_xticklabels() + thisAx2.get_yticklabels()):
                item.set_fontsize(9)
                

        if not fitError:
            confIntEWidth = abs(confIntE[0] - confIntE[1])

            results['H0Chadwick'].append(H0)
            results['EChadwick'].append(E)
            results['R2Chadwick'].append(R2)
            results['EChadwick_CIWidth'].append(confIntEWidth)
            

            results['validatedFit'].append(validatedFit)
            if validatedFit:
                results['comments'].append('ok')
            else:
                results['comments'].append('R2 < ' + str(R2CRITERION))
                
        if fitError:
            validatedFit = False
            results['H0Chadwick'].append(np.nan)
            results['EChadwick'].append(np.nan)
            results['R2Chadwick'].append(np.nan)
            results['EChadwick_CIWidth'].append(np.nan)
            results['validatedFit'].append(validatedFit)
            results['comments'].append('fitFailure')
            
        
        # (5) hysteresis (its definition may change)
        results['hysteresis'].append(hCompr[0] - hRelax[-1])
    
    if PLOT:
        archiveFig(fig1, ax1, name=results['cellID'][-1] + '_h(t)', figSubDir = 'MecaAnalysis_allCells')
        archiveFig(fig2, ax2, name=results['cellID'][-1] + '_F(h)', figSubDir = 'MecaAnalysis_allCells')
        if PLOT_SHOW:
            fig1.show()
            fig2.tight_layout()
            fig2.show()
        else:
            plt.close('all')
    
    return(results)



def createDataDict_meca(list_mecaFiles, listColumnsMeca, PLOT):
    expDf = getExperimentalConditions()
    tableDict = {}
    Nfiles = len(list_mecaFiles)
    PLOT_SHOW = (Nfiles<11)
    if not PLOT_SHOW:
        plt.ioff()
    for c in listColumnsMeca:
        tableDict[c] = []
    for f in list_mecaFiles: #[1:2]:
#         print(f)
        tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
        current_tsDF = pd.read_csv(tS_DataFilePath, ';')
        current_resultDict = analyseTimeSeries_meca(f, current_tsDF, expDf, listColumnsMeca, PLOT, PLOT_SHOW) # MAIN SUBFUNCTION
        for k in current_resultDict.keys():
            tableDict[k] += current_resultDict[k]
    plt.ion()
    return(tableDict)



def computeGlobalTable_meca(task = 'fromScratch', fileName = 'Global_MecaData', save = False, PLOT = False, \
                            listColumnsMeca=listColumnsMeca):
    """
    Compute the GlobalTable_meca from the time series data files.
    Option task='fromScratch' will analyse all the time series data files and construct a new GlobalTable from them regardless of the existing GlobalTable.
    Option task='updateExisting' will open the existing GlobalTable and determine which of the time series data files are new ones, and will append the existing GlobalTable with the data analysed from those new fils.
    listColumnsMeca have to contain all the fields of the table that will be constructed.
    """
    top = time.time()
    list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
                      if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                      and ('R40' in f))] # Change to allow different formats in the future
#     print(list_mecaFiles)
    
    if task == 'fromScratch':
        # create a dict containing the data
        tableDict = createDataDict_meca(list_mecaFiles, listColumnsMeca, PLOT) # MAIN SUBFUNCTION
        # create the dataframe from it
        meca_DF = pd.DataFrame(tableDict)
        
        # last step: now that the dataFrame is complete, one can use "compStartTimeThisDay" col to compute the start time of each compression relative to the first one done this day.
        allDates = list(meca_DF['date'].unique())
        for d in allDates:
            subDf = meca_DF.loc[meca_DF['date'] == d]
            experimentStartTime = np.min(subDf['compStartTimeThisDay'])
            meca_DF['compStartTimeThisDay'].loc[meca_DF['date'] == d] = meca_DF['compStartTimeThisDay'] - experimentStartTime
        
    elif task == 'updateExisting':
        # get existing table
        try:
            savePath = os.path.join(dataDir, (fileName + '.csv'))
            existing_meca_DF = pd.read_csv(savePath, sep=';')
        except:
            print('No existing table found')
            
        # find which of the time series files are new
        new_list_mecaFiles = []
        for f in list_mecaFiles:
            split_f = f.split('_')
            currentCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
            if currentCellID not in existing_meca_DF.cellID.values:
                new_list_mecaFiles.append(f)
                
        # create the dict with new data
        new_tableDict = createDataDict_meca(new_list_mecaFiles, listColumnsMeca, PLOT) # MAIN SUBFUNCTION
        # create the dataframe from it
        new_meca_DF = pd.DataFrame(new_tableDict)
        # fuse the existing table with the new one
        meca_DF = pd.concat([existing_meca_DF, new_meca_DF])
        
        # Reset the relative time in the day for all rows and recompute it (not elegant but simpler to write)
        meca_DF['compStartTimeThisDay'] = meca_DF['compAbsStartTime']
        allDates = list(meca_DF['date'].unique())
        for d in allDates:
            subDf = meca_DF.loc[meca_DF['date'] == d]
            experimentStartTime = np.min(subDf['compStartTimeThisDay'])
            meca_DF['compStartTimeThisDay'].loc[meca_DF['date'] == d] = meca_DF['compStartTimeThisDay'] - experimentStartTime
    
    for c in meca_DF.columns:
            if 'Unnamed' in c:
                meca_DF = meca_DF.drop([c], axis=1)
    
    if save:
        saveName = fileName + '.csv'
        savePath = os.path.join(dataDir, saveName)
        meca_DF.to_csv(savePath, sep=';')
    
    delta = time.time() - top
    print(delta)
    
    return(meca_DF)
            

    
def getGlobalTable_meca(fileName = 'Global_mecaData'):
    try:
        savePath = os.path.join(dataDir, (fileName + '.csv'))
        meca_DF = pd.read_csv(savePath, sep=';')
        print('Extracted a table with ' + str(meca_DF.shape[0]) + ' lines and ' + str(meca_DF.shape[1]) + ' columns.')
    except:
        print('No existing table found')
    for c in meca_DF.columns:
            if 'Unnamed' in c:
                meca_DF = meca_DF.drop([c], axis=1)
    
    if 'ExpDay'in meca_DF.columns:
        dateExemple = meca_DF.loc[meca_DF.index[0],'ExpDay']
        if not ('manipID' in meca_DF.columns):
            meca_DF['manipID'] = meca_DF['ExpDay'] + '_' + meca_DF['CellID'].apply(lambda x: x.split('_')[0])
            
    elif 'date'in meca_DF.columns:
        dateExemple = meca_DF.loc[meca_DF.index[0],'date']
        if not ('manipID' in meca_DF.columns):
            meca_DF['manipID'] = meca_DF['date'] + '_' + meca_DF['cellName'].apply(lambda x: x.split('_')[0])
    
    if re.match(dateFormatExcel, dateExemple):
        print('bad date')
    return(meca_DF)

In [119]:
computeGlobalTable_meca(task = 'fromScratch', fileName = 'Global_MecaData_Py', save = True, PLOT = False) # task = 'updateExisting'

Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.
11.571903467178345


,date,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,...,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments
0,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,1,1s,5.036,40650.429,0.000,79.623685,...,49.050637,76.991876,81.523497,49666.331947,0.469295,67202.105766,-14.118030,R2 > 0.9,False,R2 < 0.9
1,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,2,1s,20.249,40665.642,15.213,56.730708,...,49.050637,76.991876,69.151319,7835.426055,0.821138,3517.774613,-13.109383,R2 > 0.9,False,R2 < 0.9
2,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,3,1s,34.662,40680.055,29.626,46.999436,...,49.050637,76.991876,56.099039,16368.744911,0.789625,8702.163545,-10.464552,R2 > 0.9,False,R2 < 0.9
3,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,4,1s,49.076,40694.469,44.040,88.049320,...,49.050637,76.991876,101.313841,16640.171794,0.874840,7432.139319,-10.758911,R2 > 0.9,False,R2 < 0.9
4,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,5,1s,63.487,40708.880,58.451,54.203678,...,49.050637,76.991876,66.968358,13513.668866,0.919824,4176.032689,17.038991,R2 > 0.9,True,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,6,1s,63.627,22157.136,7591.364,180.848673,...,197.210345,56.444054,194.280385,9006.941621,0.869367,3053.504223,14.161594,R2 > 0.9,False,R2 < 0.9
2085,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,7,1s,75.543,22169.052,7603.280,234.485517,...,197.210345,56.444054,263.142485,4534.507074,0.951283,862.463087,30.461031,R2 > 0.9,True,ok
2086,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,8,1s,87.473,22180.982,7615.210,216.424678,...,197.210345,56.444054,247.631245,5922.860005,0.946598,1264.255529,12.177235,R2 > 0.9,True,ok
2087,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,9,1s,99.377,22192.886,7627.114,197.889093,...,197.210345,56.444054,220.550499,7049.102097,0.906822,1994.186725,29.488907,R2 > 0.9,True,ok


In [26]:
getGlobalTable_meca().head()

Extracted a table with 6150 lines and 20 columns.


,ExpType,ExpDay,CellID,CellName,CompNum,TpsComp,MaxIndent,MinThickness,Hysteresis,CompTime,...,SurroundingThickness,PreviousThickness,H0Chadwick,EChadwick,CiEChadwick,R2Chadwick,FitParams,Validated,Comments,manipID
0,DictyAx2-Comp,20-02-13,M1_P1_C1,20-02-13_M1_P1_C1,1,2s,172.944935,290.896640,0.468070,5.600158,...,172.291238,138.904119,531.398678,3352.518532,638.664233,0.975972,Strain100-R20.9,1,Succes!,20-02-13_M1
1,DictyAx2-Comp,20-02-13,M1_P1_C1,20-02-13_M1_P1_C1,2,4s,325.048555,340.247874,0.464737,20.967051,...,231.767587,227.523183,792.775299,1185.299024,59.056050,0.996431,Strain100-R20.9,0,Disaligned vertically,20-02-13_M1
2,DictyAx2-Comp,20-02-13,M1_P1_C1,20-02-13_M1_P1_C1,3,8s,563.627073,90.155850,0.901448,40.704944,...,192.202443,262.920701,725.884048,676.576117,74.418428,0.960774,Strain100-R20.9,0,Disaligned vertically & Has large jumps,20-02-13_M1
3,DictyAx2-Comp,20-02-13,M1_P1_C1,20-02-13_M1_P1_C1,4,2s,500.630991,321.764384,0.729998,66.117053,...,167.247232,184.655264,955.873791,685.530417,109.030000,0.983909,Strain100-R20.9,0,Disaligned vertically,20-02-13_M1
4,DictyAx2-Comp,20-02-13,M1_P1_C1,20-02-13_M1_P1_C1,5,4s,NaN,136.139589,-0.100595,81.479250,...,161.661792,162.303075,10500.987861,12.447549,33.721328,0.041727,Strain100-R20.9,0,Disaligned vertically & Has too big CI on E & ...,20-02-13_M1


In [27]:
getGlobalTable_meca('Global_MecaData_Py').head()

Extracted a table with 2089 lines and 24 columns.


,date,cellName,cellID,manipID,compNum,compDuration,compStartTime,initialThickness,minThickness,maxIndent,...,ctFieldThickness,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments
0,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,1,1s,5.036,79.623685,52.924711,26.698974,...,49.050637,76.991876,81.523497,49666.332014,0.469295,67202.105884,-14.118030,R2 > 0.9,False,R2 < 0.9
1,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,2,1s,20.249,56.730708,10.178800,46.551908,...,49.050637,76.991876,69.151319,7835.426036,0.821138,3517.774598,-13.109383,R2 > 0.9,False,R2 < 0.9
2,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,3,1s,34.662,46.999436,15.165123,31.834313,...,49.050637,76.991876,56.099039,16368.744932,0.789625,8702.163508,-10.464552,R2 > 0.9,False,R2 < 0.9
3,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,4,1s,49.076,88.049320,51.951057,36.098263,...,49.050637,76.991876,101.313841,16640.171782,0.874840,7432.139272,-10.758911,R2 > 0.9,False,R2 < 0.9
4,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,5,1s,63.487,54.203678,19.679992,34.523685,...,49.050637,76.991876,66.968358,13513.668871,0.919824,4176.032701,17.038991,R2 > 0.9,True,ok


### Fluorescence

In [28]:
def getFluoData(save = False):
    # Getting the table
    fluoDataFile = 'FluoQuantification.csv'
    fluoDataFilePath = os.path.join(dataDir, fluoDataFile)
    fluoDF = pd.read_csv(fluoDataFilePath, sep=';',header=0)
    print('Extracted a table with ' + str(fluoDF.shape[0]) + ' lines and ' + str(fluoDF.shape[1]) + ' columns.')
    # Cleaning the table
    try:
        for c in fluoDF.columns:
            if 'Unnamed' in c:
                fluoDF = fluoDF.drop([c], axis=1)
        
    except:
        print('Unexpected bug with the cleaning step')

    if save:
        saveName = 'FluoQuantification.csv'
        savePath = os.path.join(dataDir, saveName)
        fluoDF.to_csv(savePath, sep=';')

    
    return(fluoDF)

In [29]:
getFluoData().head()

Extracted a table with 108 lines and 2 columns.


,cellID,meanFluoPeakAmplitude
0,21-01-18_M1_P1_C5,1798.786253
1,21-01-18_M1_P1_C6,768.192944
2,21-01-18_M1_P1_C7,686.701203
3,21-01-18_M1_P1_C8,1655.534620
4,21-01-18_M1_P1_C9,187.560043


### Comparison meca tables from matlab vs python

* Comparison - Per compression

Problems with an offset in the indexation of compression in the matlab table

In [30]:
# Comparison of values obtained with the python code vs matlab

table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()

mecaPy = getGlobalTable_meca('Global_MecaData_Py')
mecaPy = pd.merge(table_ExpConditions, mecaPy, how="inner", on='manipID')
mecaPy = pd.merge(mecaPy, table_fluo, how="left", on='cellID')
mecaPy = mecaPy.sort_values(["cellID", "compNum"], ascending = (True, True),ignore_index=True)
# mecaPy = mecaPy.loc[(mecaPy['cell subtype'] == 'aSFL') | (mecaPy['cell subtype'] == 'aSFL-6FP')]
mecaMat = getGlobalTable_meca('Global_MecaData')
mecaMat = pd.merge(table_ExpConditions, mecaMat, how="inner", on='manipID')
mecaMat = pd.merge(mecaMat, table_fluo, how="left", left_on='CellName', right_on='cellID')
mecaMat = mecaMat.sort_values(["CellName", "CompNum"], ascending = (True, True),ignore_index=True)

FiltersPy = [(mecaMat['Validated'] == True), (mecaPy['validatedFit'] == True)]
filterGlobal = np.ones((mecaPy.shape[0]), dtype=bool)
for fltr in FiltersPy:
    filterGlobal = filterGlobal & fltr
mecaPyF = mecaPy.loc[filterGlobal]
mecaPyF_light = mecaPyF[['compNum','H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
mecaPyF.tail()
np.sum(mecaPyF['validatedFit'].values)
mecaPyF

Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.
Extracted a table with 108 lines and 2 columns.
Extracted a table with 2089 lines and 24 columns.
Extracted a table with 6150 lines and 20 columns.


,date_x,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,ctFieldFluctuAmpli,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,hysteresis,critFit,validatedFit,comments_y,meanFluoPeakAmplitude
4,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,76.991876,66.968358,13513.668871,0.919824,4176.032701,17.038991,R2 > 0.9,True,ok,NaN
5,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,76.991876,175.663940,6585.285806,0.979245,1047.491207,8.896213,R2 > 0.9,True,ok,NaN
6,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,76.991876,181.761205,4342.729853,0.947873,1078.223347,-14.628913,R2 > 0.9,True,ok,NaN
7,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,51.512029,173.345993,26847.070872,0.980815,4292.282931,8.845257,R2 > 0.9,True,ok,NaN
8,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,51.512029,147.922861,28190.336376,0.963097,7031.906292,8.842108,R2 > 0.9,True,ok,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,56.444054,443.990436,2331.437562,0.992319,180.652597,114.976514,R2 > 0.9,True,ok,77.71709
2085,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,56.444054,263.142485,4534.507073,0.951283,862.463086,30.461031,R2 > 0.9,True,ok,77.71709
2086,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,56.444054,247.631245,5922.859998,0.946598,1264.255526,12.177235,R2 > 0.9,True,ok,77.71709
2087,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,56.444054,220.550499,7049.102095,0.906822,1994.186717,29.488907,R2 > 0.9,True,ok,77.71709


In [31]:
FiltersMat = [(mecaPy['validatedFit'] == True), (mecaMat['Validated'] == True)]
filterGlobal = np.ones((mecaMat.shape[0]), dtype=bool)
for fltr in FiltersPy:
    filterGlobal = filterGlobal & fltr
mecaMatF = mecaMat.loc[filterGlobal]
mecaMatF = mecaMatF.rename(columns={'CompNum': 'compNum', 'CiEChadwick': 'EChadwick_CIWidth'})
mecaMatF_light = mecaMatF[['compNum','H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
# mecaMat_light.index = [i for i in range(Bbis.shape[0])]
mecaMatF

,date,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,PreviousThickness,H0Chadwick,EChadwick,EChadwick_CIWidth,R2Chadwick,FitParams,Validated,Comments,cellID,meanFluoPeakAmplitude
4,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,56.189574,67.310001,13047.464921,4224.648359,0.934253,Strain100-R20.9,1,Succes!,NaN,NaN
5,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,53.414657,169.713331,7321.384845,1322.656518,0.981516,Strain100-R20.9,1,Succes!,NaN,NaN
6,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,52.650556,173.331245,4835.754348,1407.552795,0.951041,Strain100-R20.9,1,Succes!,NaN,NaN
7,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,145.580764,171.740996,28523.413066,6352.699750,0.978123,Strain100-R20.9,1,Succes!,NaN,NaN
8,20-08-04,M1,compressions and constant field,none,BSA coated glass,100X,15.8,oil,0.875,1.15,...,114.965682,145.874447,30533.680801,8024.926067,0.965118,Strain100-R20.9,1,Succes!,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2083,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,228.596106,432.103180,2545.995167,262.446743,0.990609,Strain100-R20.9,1,Succes!,21-06-17_M2_P1_C9,77.71709
2085,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,197.122107,235.841794,6566.687801,2536.764980,0.922079,Strain100-R20.9,1,Succes!,21-06-17_M2_P1_C9,77.71709
2086,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,214.328291,229.116584,7955.408651,2429.554237,0.947268,Strain100-R20.9,1,Succes!,21-06-17_M2_P1_C9,77.71709
2087,21-06-17,M2,compressions,none,20um fibronectin discs,100X,15.8,oil,0.875,1.10,...,196.664406,172.608654,21957.744015,8277.352195,0.926790,Strain100-R20.9,1,Succes!,21-06-17_M2_P1_C9,77.71709


In [32]:
mecaCompare = (mecaPyF_light-mecaMatF_light)/mecaMatF_light
Th = 0.1
mecaCompare['Evaries'] = (np.abs(mecaCompare.EChadwick) > Th)
mecaCompare['H0varies'] = (np.abs(mecaCompare.H0Chadwick) > Th)
mecaCompare['cellID'] = mecaPyF['cellID']
mecaCompare['compNum'] = mecaPyF['compNum']
mecaCompare['cellID2'] = mecaMatF['CellName']
mecaCompare['compNum2'] = mecaMatF['compNum']
mecaCompare = mecaCompare[['H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth','Evaries','H0varies','cellID','compNum','cellID2','compNum2']]
print('Nb of signif changes in E : {:.0f} / {:.0f} ; in H0 : {:.0f} / {:.0f}'.format(np.sum(mecaCompare['Evaries']), mecaCompare.shape[0], np.sum(mecaCompare['H0varies']), mecaCompare.shape[0]))
mecaCompare

Nb of signif changes in E : 632 / 1313 ; in H0 : 33 / 1313


,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth,Evaries,H0varies,cellID,compNum,cellID2,compNum2
4,-0.005076,0.035731,-0.015444,-0.011508,False,False,20-08-04_M1_P1_C5,5,20-08-04_M1_P1_C5,5
5,0.035063,-0.100541,-0.002314,-0.208040,True,False,20-08-04_M1_P1_C5,6,20-08-04_M1_P1_C5,6
6,0.048635,-0.101954,-0.003332,-0.233973,True,False,20-08-04_M1_P1_C5,7,20-08-04_M1_P1_C5,7
7,0.009345,-0.058771,0.002752,-0.324337,False,False,20-08-04_M1_P1_C6,1,20-08-04_M1_P1_C6,1
8,0.014042,-0.076746,-0.002094,-0.123742,False,False,20-08-04_M1_P1_C6,2,20-08-04_M1_P1_C6,2
...,...,...,...,...,...,...,...,...,...,...
2083,0.027510,-0.084273,0.001726,-0.311660,False,False,21-06-17_M2_P1_C9,5,21-06-17_M2_P1_C9,5
2085,0.115758,-0.309468,0.031672,-0.660015,True,True,21-06-17_M2_P1_C9,7,21-06-17_M2_P1_C9,7
2086,0.080809,-0.255493,-0.000708,-0.479635,True,False,21-06-17_M2_P1_C9,8,21-06-17_M2_P1_C9,8
2087,0.277749,-0.678970,-0.021545,-0.759079,True,True,21-06-17_M2_P1_C9,9,21-06-17_M2_P1_C9,9


* Comparison - Per cell

In [33]:
# Comparison of values obtained with the python code vs matlab

table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()

mecaPy = getGlobalTable_meca('Global_MecaData_Py')
mecaPy = pd.merge(table_ExpConditions, mecaPy, how="inner", on='manipID')
mecaPy = pd.merge(mecaPy, table_fluo, how="left", on='cellID')
mecaPy = mecaPy.sort_values(["cellID", "compNum"], ascending = (True, True),ignore_index=True)
# mecaPy = mecaPy.loc[(mecaPy['cell subtype'] == 'aSFL') | (mecaPy['cell subtype'] == 'aSFL-6FP')]
mecaMat = getGlobalTable_meca('Global_MecaData')
mecaMat = pd.merge(table_ExpConditions, mecaMat, how="inner", on='manipID')
mecaMat = pd.merge(mecaMat, table_fluo, how="left", left_on='CellName', right_on='cellID')
mecaMat = mecaMat.sort_values(["CellName", "CompNum"], ascending = (True, True),ignore_index=True)

FiltersPy = [(mecaMat['Validated'] == True), (mecaPy['validatedFit'] == True)]
filterGlobal = np.ones((mecaPy.shape[0]), dtype=bool)
for fltr in FiltersPy:
    filterGlobal = filterGlobal & fltr
mecaPyF = mecaPy.loc[filterGlobal]

cellID = 'cellID'
group = mecaPyF.groupby(cellID)
dictAggMean = getDictAggMean(mecaPyF)
mecaPyF_perCell = group.agg(dictAggMean)

mecaPyF_perCell_light = mecaPyF_perCell[['H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
mecaPyF_perCell_light.head()

Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.
Extracted a table with 108 lines and 2 columns.
Extracted a table with 2089 lines and 24 columns.
Extracted a table with 6150 lines and 20 columns.


,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth
cellID,,,,
20-08-04_M1_P1_C5,141.464501,8147.228177,0.948981,2100.582419
20-08-04_M1_P1_C6,167.803404,30218.993975,0.961608,7583.446398
20-08-04_M1_P1_C7,308.085927,6044.947440,0.990187,678.062769
20-08-04_M2_P1_C5,63.157911,14755.258112,0.972031,2546.925419
20-08-05_M1_P1_C3,73.440669,21033.636202,0.952709,5264.693713


In [34]:
FiltersMat = [(mecaPy['validatedFit'] == True), (mecaMat['Validated'] == True)]
filterGlobal = np.ones((mecaMat.shape[0]), dtype=bool)
for fltr in FiltersPy:
    filterGlobal = filterGlobal & fltr
mecaMatF = mecaMat.loc[filterGlobal]
mecaMatF = mecaMatF.rename(columns={'CompNum': 'compNum', 'CiEChadwick': 'EChadwick_CIWidth'})

cellID = 'CellName'
group = mecaMatF.groupby(cellID)
dictAggMean = getDictAggMean(mecaMatF)
mecaMatF_perCell = group.agg(dictAggMean)


mecaMatF_perCell_light = mecaMatF_perCell[['H0Chadwick','EChadwick','R2Chadwick','EChadwick_CIWidth']]
# mecaMat_light.index = [i for i in range(Bbis.shape[0])]
# mecaMatF_perCell
mecaMatF_perCell_light.head()

,H0Chadwick,EChadwick,R2Chadwick,EChadwick_CIWidth
CellName,,,,
20-08-04_M1_P1_C5,136.784859,8401.534705,0.955603,2318.285891
20-08-04_M1_P1_C6,169.217609,28079.073974,0.971391,6766.511703
20-08-04_M1_P1_C7,311.117498,5902.871686,0.994095,603.446684
20-08-04_M2_P1_C5,64.183088,14183.797788,0.980552,2158.348466
20-08-05_M1_P1_C3,75.598024,18736.501327,0.967227,3980.988452


In [35]:
mecaCompare = (mecaPyF_perCell_light-mecaMatF_perCell_light)/mecaMatF_perCell_light
listE = []
listH0 = []
for i in range(1,20):
    Th = 0.05*i
    mecaCompare['Evaries'] = (np.abs(mecaCompare.EChadwick) > Th)
    mecaCompare['H0varies'] = (np.abs(mecaCompare.H0Chadwick) > Th)
    mecaCompare['cellID'] = mecaPyF_perCell['cellID']
    mecaCompare['cellID2'] = mecaMatF_perCell['CellName']
    listE.append(np.sum(mecaCompare['Evaries']))
    listH0.append(np.sum(mecaCompare['H0varies']))
    print('For T = {:.0f}%, E : {:.0f}/{:.0f} ; H0 : {:.0f}/{:.0f}'\
          .format(Th*100, np.sum(mecaCompare['Evaries']), mecaCompare.shape[0], np.sum(mecaCompare['H0varies']), mecaCompare.shape[0]))
# mecaCompare #.loc[mecaCompare['Evaries']]

For T = 5%, E : 128/174 ; H0 : 12/174
For T = 10%, E : 81/174 ; H0 : 2/174
For T = 15%, E : 43/174 ; H0 : 2/174
For T = 20%, E : 26/174 ; H0 : 1/174
For T = 25%, E : 15/174 ; H0 : 1/174
For T = 30%, E : 7/174 ; H0 : 0/174
For T = 35%, E : 4/174 ; H0 : 0/174
For T = 40%, E : 1/174 ; H0 : 0/174
For T = 45%, E : 0/174 ; H0 : 0/174
For T = 50%, E : 0/174 ; H0 : 0/174
For T = 55%, E : 0/174 ; H0 : 0/174
For T = 60%, E : 0/174 ; H0 : 0/174
For T = 65%, E : 0/174 ; H0 : 0/174
For T = 70%, E : 0/174 ; H0 : 0/174
For T = 75%, E : 0/174 ; H0 : 0/174
For T = 80%, E : 0/174 ; H0 : 0/174
For T = 85%, E : 0/174 ; H0 : 0/174
For T = 90%, E : 0/174 ; H0 : 0/174
For T = 95%, E : 0/174 ; H0 : 0/174


### Data import & DataFrame formatting

In [138]:
pd.set_option('max_columns', None)
pd.reset_option('max_columns')
pd.set_option('max_rows', None)
pd.reset_option('max_rows')

In [139]:
# getExperimentalConditions().head()
# getGlobalTable_ctField().head()
# getGlobalTable_meca().head()
# getFluoData().head()

In [140]:
# GlobalTable_ctField

GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_ctField = pd.merge(GlobalTable_ctField, table_fluo, how="left", on='cellID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_ctField.shape[0]) + ' lines and ' + str(GlobalTable_ctField.shape[1]) + ' columns.')

GlobalTable_ctField.head()

Extracted a table with 40 lines and 12 columns.
Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.
Extracted a table with 108 lines and 2 columns.
Merged table has 40 lines and 33 columns.


,date_x,manip,experimentType,drug,substrate,objective magnification,scale pixel per um,objective immersion,optical index correction,magnetic field correction,...,cellID,duration,medianRawB,medianThickness,1stDThickness,9thDThickness,fluctuAmpli,R2_polyFit,validated,meanFluoPeakAmplitude
0,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,...,21-02-10_M1_P1_C1,393.550,6.0036,183.232388,103.105084,276.671473,173.566389,0.222994,1.0,NaN
1,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,...,21-02-10_M1_P1_C2,326.388,6.0036,-58.229968,-147.203982,41.303068,188.507051,0.228033,0.0,NaN
2,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,...,21-02-10_M1_P1_C3,327.997,6.0036,204.963636,123.738478,277.782516,154.044038,0.402168,1.0,NaN
3,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,...,21-02-10_M1_P1_C4,326.929,6.0036,203.321336,165.769700,282.439284,116.669584,0.261129,1.0,NaN
4,21-02-10,M1,constant field,none,20um fibronectin discs,100X,15.8,oil,0.875,1.2,...,21-02-10_M1_P1_C5,260.260,6.0036,231.480125,169.220846,301.601770,132.380924,0.248455,1.0,NaN


In [141]:
# GlobalTable_meca

GlobalTable_meca = getGlobalTable_meca()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca = pd.merge(GlobalTable_meca, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca = pd.merge(GlobalTable_meca, table_fluo, how="left", left_on='CellName', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca.shape[0]) + ' lines and ' + str(GlobalTable_meca.shape[1]) + ' columns.')

GlobalTable_meca.tail()

Extracted a table with 6150 lines and 20 columns.
Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.
Extracted a table with 108 lines and 2 columns.
Merged table has 2089 lines and 43 columns.


,ExpType,ExpDay,CellID,CellName,CompNum,TpsComp,MaxIndent,MinThickness,Hysteresis,CompTime,...,bead type,bead diameter,normal field,ramp field,compression duration,with fluo images,bacteria,comments,cellID,meanFluoPeakAmplitude
2084,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,6,1s,NaN,NaN,NaN,NaN,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,21-06-17_M2_P1_C12,115.022849
2085,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,7,1s,NaN,NaN,NaN,NaN,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,21-06-17_M2_P1_C12,115.022849
2086,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,8,1s,NaN,NaN,NaN,NaN,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,21-06-17_M2_P1_C12,115.022849
2087,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,9,1s,64.512472,118.090149,-0.121251,99.541673,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,21-06-17_M2_P1_C12,115.022849
2088,3T3aSFL-A8_nodrug,21-06-17,M2_P1_C12,21-06-17_M2_P1_C12,10,1s,73.110975,132.492617,0.357010,111.458527,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,21-06-17_M2_P1_C12,115.022849


In [142]:
# GlobalTable_mecaBis

GlobalTable_meca_Py = getGlobalTable_meca('Global_MecaData_Py')

table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca_Py = pd.merge(GlobalTable_meca_Py, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca_Py = pd.merge(GlobalTable_meca_Py, table_fluo, how="left", left_on='cellID', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca_Py.shape[0]) + ' lines and ' + str(GlobalTable_meca_Py.shape[1]) + ' columns.')

Extracted a table with 2089 lines and 26 columns.
Extracted a table with 34 lines and 21 columns.
optical index correction already in Float64 type.
Extracted a table with 108 lines and 2 columns.
Merged table has 2089 lines and 46 columns.


In [143]:
GlobalTable_meca_Py.tail()

,date_x,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,...,cell subtype,bead type,bead diameter,normal field,ramp field,compression duration,with fluo images,bacteria,comments_y,meanFluoPeakAmplitude
2084,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,6,1s,63.627,22157.136,7591.364,180.848673,...,aSFL-A8,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709
2085,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,7,1s,75.543,22169.052,7603.280,234.485517,...,aSFL-A8,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709
2086,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,8,1s,87.473,22180.982,7615.210,216.424678,...,aSFL-A8,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709
2087,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,9,1s,99.377,22192.886,7627.114,197.889093,...,aSFL-A8,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709
2088,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,10,1s,111.308,22204.817,7639.045,163.094576,...,aSFL-A8,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709


In [144]:
# Experiment counter - Matlab table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'CellName'
GlobalTable_meca_CountComp = GlobalTable_meca.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'CellName'
GlobalTable_meca_CountCompOK = GlobalTable_meca[GlobalTable_meca['Validated'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'CellName'
group = GlobalTable_meca.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

Count cells - ctField  \
cell type cell subtype bead type drug       substrate                                       
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                     20   
                                            BSA coated glass                            0   
                                 none       20um fibronectin discs                     20   
                                            BSA coated glass                            0   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                      0   
                                 none       20um fibronectin discs                      0   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                      0   
                                 none       20um fibronectin discs                      0   

                                                                    Count cells - meca  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                  28   
                                            BSA coated glass                        16   
                                 none       20um fibronectin discs                  32   
                                            BSA coated glass                        13   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                  26   
                                 none       20um fibronectin discs                  29   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                  25   
                                 none       20um fibronectin discs                  23   

                                                                    Count compressions  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                 352   
                                            BSA coated glass                       161   
                                 none       20um fibronectin discs                 392   
                                            BSA coated glass                       117   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                 254   
                                 none       20um fibronectin discs                 293   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                 260   
                                 none       20um fibronectin discs                 260   

                                                                    Count OK compressions  
cell type cell subtype bead type drug       substrate                                      
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                    216  
                                            BSA coated glass                          105  
                                 none       20um fibronectin discs                    314  
                                            BSA coated glass                           60  
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                    161  
                                 none       20um fibronectin discs                    160  
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                    168  
                                 none       20um fibronectin discs                    183

In [145]:
# Experiment counter - Python table

# count ct field cells
cellID = 'cellID'
GlobalTable_ctField_CountCell = GlobalTable_ctField.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_ctField_CountCell = GlobalTable_ctField_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - ctField'})

# count meca compressions
cellID = 'cellID'
GlobalTable_meca_CountComp = GlobalTable_meca_Py.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountComp = GlobalTable_meca_CountComp.loc[:, [cellID]].rename(columns={cellID : 'Count compressions'})

# count valid meca compressions
cellID = 'cellID'
GlobalTable_meca_CountCompOK = GlobalTable_meca_Py[GlobalTable_meca_Py['validatedFit'] == True].groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCompOK = GlobalTable_meca_CountCompOK.loc[:, [cellID]].rename(columns={cellID : 'Count OK compressions'})

# count meca cells
cellID = 'cellID'
group = GlobalTable_meca_Py.groupby(cellID)
dictAggMean = getDictAggMean(GlobalTable_meca_Py)
GlobalTable_meca_perCell = group.agg(dictAggMean)
GlobalTable_meca_CountCell = GlobalTable_meca_perCell.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_CountCell = GlobalTable_meca_CountCell.loc[:, [cellID]].rename(columns={cellID : 'Count cells - meca'})


# Fuse all the previous tables
GlobalTable_CountAll = pd.concat([GlobalTable_ctField_CountCell, GlobalTable_meca_CountCell, GlobalTable_meca_CountComp, GlobalTable_meca_CountCompOK], axis=1)
GlobalTable_CountAll = GlobalTable_CountAll.fillna(0)
GlobalTable_CountAll = GlobalTable_CountAll.loc[:,:].astype(int)
GlobalTable_CountAll

Count cells - ctField  \
cell type cell subtype bead type drug       substrate                                       
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                     20   
                                            BSA coated glass                            0   
                                 none       20um fibronectin discs                     20   
                                            BSA coated glass                            0   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                      0   
                                 none       20um fibronectin discs                      0   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                      0   
                                 none       20um fibronectin discs                      0   

                                                                    Count cells - meca  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                  28   
                                            BSA coated glass                        16   
                                 none       20um fibronectin discs                  32   
                                            BSA coated glass                        13   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                  26   
                                 none       20um fibronectin discs                  29   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                  25   
                                 none       20um fibronectin discs                  23   

                                                                    Count compressions  \
cell type cell subtype bead type drug       substrate                                    
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                 352   
                                            BSA coated glass                       161   
                                 none       20um fibronectin discs                 392   
                                            BSA coated glass                       117   
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                 254   
                                 none       20um fibronectin discs                 293   
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                 260   
                                 none       20um fibronectin discs                 260   

                                                                    Count OK compressions  
cell type cell subtype bead type drug       substrate                                      
3T3       aSFL         M450      doxycyclin 20um fibronectin discs                    217  
                                            BSA coated glass                          105  
                                 none       20um fibronectin discs                    323  
                                            BSA coated glass                           60  
          aSFL-6FP     M450      doxycyclin 20um fibronectin discs                    164  
                                 none       20um fibronectin discs                    157  
          aSFL-A8      M450      doxycyclin 20um fibronectin discs                    171  
                                 none       20um fibronectin discs                    188

In [146]:
# Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8')]
# GlobalTable_meca_PyF = GlobalTable_meca_Py
# for fltr in Filters:
#         GlobalTable_meca_PyF = GlobalTable_meca_PyF.loc[fltr]
# # GlobalTable_mecaBis
# cellID = 'cellID'
# group = GlobalTable_meca_PyF.groupby(cellID)
# dictAggMean = getDictAggMean(GlobalTable_meca_PyF)
# GlobalTable_meca_PyF_perCell = group.agg(dictAggMean)
# GlobalTable_meca_PyF_perCell

## Plots

### Functions

In [45]:
Styles = {''} # Project of automatic formatting according to the type of data

These functions use matplotlib.pyplot and seaborn libraries to display 1D categorical or 2D plots

In [60]:
def D1Plot(data, CondCol=[],Parameters=[],Filters=[],Boxplot=True,AvgPerCell=False,cellID='cellID', co_order=[],\
           stats=True,statMethod='Mann-Whitney',box_pairs=[],figSizeFactor = 1,markerSizeFactor=1):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
    NCond = len(CondCol)    
    if NCond == 1:
        CondCol = CondCol[0]
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean)
        
    data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if len(co_order) == 0:
        co_order = Conditions
        
    fig, ax = plt.subplots(1, NPlots, figsize = (5*NPlots*NCond*figSizeFactor,5))
    markerSize = 5*markerSizeFactor
    
    if NPlots > 1:
        for k in range(NPlots):
            
            if Parameters[k] == 'EChadwick':
                ax[k].set_yscale('log')
                

            if Boxplot:
                sns.boxplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], 
                            color='w', linewidth = 2, width = 0.5, showfliers = False, order= co_order)
                # data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],showfliers = False) # linewidth = 2, width = 0.5
            
            if stats:
                if len(box_pairs) == 0:
                    box_pairs = makeBoxPairs(co_order)
                addStat(ax[k], data_filtered, box_pairs, Parameters[k], CondCol, test = statMethod)
                # add_stat_annotation(ax[k], x=CondCol, y=Parameters[k], data=data_filtered,box_pairs = box_pairs,test=statMethod, text_format='star',loc='inside', verbose=2)
                    
            sns.swarmplot(x=CondCol, y=Parameters[k], data=data_filtered, ax=ax[k], 
                          size=markerSize, edgecolor='k',linewidth = 1*markerSizeFactor, order= co_order)
            
            ax[k].set_xlabel('')
            ax[k].set_ylabel(Parameters[k])
            ax[k].tick_params(axis='x', labelrotation = 10)
            ax[k].yaxis.grid(True)           
            
    else:
        if Parameters[0] == 'EChadwick':
            ax.set_yscale('log')
                
        if Boxplot:
            sns.boxplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, 
                        color='w', linewidth = 2, width = 0.5, showfliers = False, order= co_order)
            # data_filtered.boxplot(column=Parameters[k], by = CondCol, ax=ax[k],showfliers = False) # linewidth = 2, width = 0.5

        if stats:
            if len(box_pairs) == 0:
                box_pairs = makeBoxPairs(co_order)
            addStat(ax, data_filtered, box_pairs, Parameters[0], CondCol, test = statMethod)
            # add_stat_annotation(ax, x=CondCol, y=Parameters[0], data=data_filtered,box_pairs = box_pairs,test=statMethod, text_format='star',loc='inside', verbose=2)

        sns.swarmplot(x=CondCol, y=Parameters[0], data=data_filtered, ax=ax, 
                      size=markerSize, edgecolor='k',linewidth = 1*markerSizeFactor, order= co_order)
    
        ax.set_xlabel('')
        ax.set_ylabel(Parameters[0])
        ax.tick_params(axis='x', labelrotation = 10)
        ax.yaxis.grid(True)

    return(fig, ax)

In [91]:
def D2Plot(data, XCol='',YCol='',CondCol='',Filters=[], cellID='cellID', AvgPerCell=False, modelFit=False, modelType='y=ax+b',\
           figSizeFactor = 1):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
    
    NCond = len(CondCol)    
    if NCond == 1:
        CondCol = CondCol[0]
    elif NCond > 1:
        newColName = ''
        for i in range(NCond):
            newColName += CondCol[i]
            newColName += ' & '
        newColName = newColName[:-3]
        data_filtered[newColName] = ''
        for i in range(NCond):
            data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
            data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
        CondCol = newColName
    
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
        
    Conditions = list(data_filtered[CondCol].unique())
    fig, ax = plt.subplots(1, 1, figsize = (8*figSizeFactor,5))
    markerSize = 5
    
    if modelFit:
        # Tweak the style cycle to plot for each condition: the points ('o') and then the fit ('-') with the same color.
        ncustom_color_list = list(np.array([new_color_list, new_color_list]).flatten(order='F'))
        # if new_color_list was ['red', 'green', blue'], custom_color_list is now ['red', 'red', 'green', 'green', blue', blue']
        cc = cycler(color=ncustom_color_list)
        ax.set_prop_cycle(cc)
    
    for c in Conditions:
        Xraw = data_filtered[data_filtered[CondCol] == c][XCol].values
        Yraw = data_filtered[data_filtered[CondCol] == c][YCol].values
        XYraw = np.array([Xraw,Yraw]).T
        XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
        X, Y = XY[:,0], XY[:,1]
        if len(X) > 0:
            eqnText = ''

            if modelFit:
                print('Fitting condition: ' + c + ' with model ' + modelType)
                if modelType == 'y=ax+b':
                    params, results = fitLine(X, Y) # Y=a*X+b ; params[0] = b,  params[1] = a
                    pval = results.pvalues[1] # pvalue on the param 'a'
                    eqnText += " ; Y = {:.1f} X + {:.1f}".format(params[1], params[0])
                    eqnText += " ; p-val = {:.2e}".format(pval)
                    print("Y = {:.5} X + {:.5}".format(params[1], params[0]))
                    print("p-value on the 'a' coefficient: {:.4e}".format(pval))
                    print('\n')
                    fitY = params[1]*X + params[0]
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1')

                elif modelType == 'y=A*exp(kx)':
                    params, results = fitLine(X, np.log(Y)) # Y=a*X+b ; params[0] = b,  params[1] = a
                    pval = results.pvalues[1] # pvalue on the param 'a'
                    eqnText += " ; Y = {:.1f}*exp({:.1f}*X)".format(params[0], params[1])
                    eqnText += " ; p-val = {:.2e}".format(pval)
                    print("Y = {:.5}*exp({:.5}*X)".format(np.exp(params[0]), params[1]))
                    print("p-value on the 'k' coefficient: {:.4e}".format(pval))
                    print('\n')
                    fitY = np.exp(params[0])*np.exp(params[1]*X)
                    imin = np.argmin(X)
                    imax = np.argmax(X)
                    ax.plot([X[imin],X[imax]], [fitY[imin],fitY[imax]], '--', lw = '1')
                    
            ax.plot(X, Y, 'o', markersize = markerSize, markeredgecolor='k', markeredgewidth = 1, label=c + eqnText)
            
            
    ax.set_xlabel(XCol)
    ax.set_xlim([min(0,1.1*np.min(data_filtered[XCol])), 1.1*np.max(data_filtered[XCol])])
    ax.set_ylabel(YCol)
    ax.set_ylim([min(0,1.1*np.min(data_filtered[YCol])), 1.1*np.max(data_filtered[YCol])])
    ax.legend(loc='upper left')
    return(fig, ax)

These functions use the Bokeh library to display 1D categorical or 2D plots with interactive plots. They are less flexible but can be nice to explore the data set since you can display the cellID which is the source of each point by passing your pointer over it.

In [48]:
def D1PlotInteractive(data, CondCol='',Parameters=[],Filters=[],AvgPerCell=False,cellID='cellID'):
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
#     print(data_filtered[cellID])
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
#     return(data_filtered)
    
    NPlots = len(Parameters)
    Conditions = list(data_filtered[CondCol].unique())
    if NPlots > 1:
        plots = []
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )        
        
        for k in range(NPlots):
            hover = HoverTool(
                tooltips=[
                    ('Cell ID', "@"+cellID),
                    (Parameters[k], "@"+Parameters[k]),
                ]
            )
            index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
            p = figure(plot_width=450, plot_height=500, tools=[hover], title="InteractivePlot") # 
            p.circle('X_jitter', Parameters[k], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
            # Format
            p.x_range = Range1d(0, NCond+1)
            p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[k]]))
            p.xaxis.ticker = [i for i in range(1,NCond+1)]
            p.xaxis.major_label_overrides = dictTicks
            p.xaxis.axis_label = CondCol
            p.xaxis.axis_label_text_font_size = '18pt'
            p.xaxis.major_label_text_font_size = '16pt'
            p.yaxis.axis_label = Parameters[k]
            p.yaxis.axis_label_text_font_size = '18pt'
            p.yaxis.major_label_text_font_size = '16pt'
            
            plots.append(p)
            
        p = gridplot(plots, ncols=2, toolbar_location=None)
        
        
    else:
        hover = HoverTool(
            tooltips=[
                ('Cell ID', "@"+cellID),
                (Parameters[0], "@"+Parameters[0]),
            ]
        )
        
        NCond = len(Conditions)
        data_filtered['X'] = 0
        data_filtered['X_jitter'] = 0.
        dictTicks = {}
        for i in range(NCond):
            mask = data_filtered[CondCol] == Conditions[i]
            data_filtered.loc[mask, 'X'] = i+1
            dictTicks[i+1] = Conditions[i]
        for i in data_filtered.index:
            data_filtered.loc[i, 'X_jitter'] = data_filtered.loc[i, 'X'] + 0.4*(np.random.rand(1)[0]-0.5)
        source = ColumnDataSource(
            data=data_filtered[[cellID]+[CondCol]+Parameters+['X','X_jitter']]
        )
        index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
        TOOLS = "hover,pan,box_zoom,wheel_zoom,reset,save,help"
        p = figure(plot_width=500, plot_height=500, tools=TOOLS, title="InteractivePlot") # 
        p.circle('X_jitter', Parameters[0], size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
        # Format
        p.x_range = Range1d(0, NCond+1)
        p.y_range = Range1d(min(0,1.1*np.min(data_filtered[Parameters[0]])), 1.1*np.max(data_filtered[Parameters[0]]))
        p.xaxis.ticker = [i for i in range(1,NCond+1)]
        p.xaxis.major_label_overrides = dictTicks
        p.xaxis.axis_label = CondCol
        p.xaxis.axis_label_text_font_size = '18pt'
        p.xaxis.major_label_text_font_size = '16pt'
        p.yaxis.axis_label = Parameters[0]
        p.yaxis.axis_label_text_font_size = '18pt'
        p.yaxis.major_label_text_font_size = '16pt'
    return(p)

In [49]:
def D2PlotInteractive(data, XCol='',YCol='',CondCol='',Filters=[], cellID='cellID',AvgPerCell=False):
    
    data_filtered = data
    for fltr in Filters:
        data_filtered = data_filtered.loc[fltr]
        
    if AvgPerCell:
        group = data_filtered.groupby(cellID)
        dictAggMean = getDictAggMean(data_filtered)
        data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
        data_filtered.reset_index(level=0, inplace=True)
    
    Conditions = list(data_filtered[CondCol].unique())

    NCond = len(Conditions)
    dictTicks = {}
    for i in range(NCond):
        dictTicks[i+1] = Conditions[i]
    
    source = ColumnDataSource(
        data=data_filtered[[cellID,CondCol,XCol,YCol]]
    )
    
    hover = HoverTool(
        tooltips=[
            ('Cell ID', "@"+cellID),
            (XCol, "@"+XCol),
            (YCol, "@"+YCol),
            (CondCol, "@"+CondCol),
        ]
    )
    
    index_cmap = factor_cmap(CondCol, palette=Category10[10], factors=sorted(data_filtered[CondCol].unique()), end=1)
    TOOLS = "pan,box_zoom,wheel_zoom,reset,save,help"
    p = figure(plot_width=900, plot_height=500, tools=TOOLS, title="InteractivePlot",toolbar_location="below") # 
    p.circle(XCol, YCol, size=8, alpha = 0.6, source=source,fill_color=index_cmap,line_color='black')
    p.add_tools(hover)
    # Format
    p.x_range = Range1d(0, 1.1*np.max(data_filtered[XCol]))
    p.y_range = Range1d(0, 1.1*np.max(data_filtered[YCol]))
    p.xaxis.axis_label = XCol
    p.xaxis.axis_label_text_font_size = '18pt'
    p.xaxis.major_label_text_font_size = '16pt'
    p.yaxis.axis_label = YCol
    p.yaxis.axis_label_text_font_size = '18pt'
    p.yaxis.major_label_text_font_size = '16pt'
    return(p)

Other subfunctions useful to plot

In [98]:
def makeOrder(*args):
    order = []
    listeTuple = list(itertools.product(*args, repeat=1))
    for tup in listeTuple:
        tmpText = ''
        for word in tup:
            tmpText += word
            tmpText += ' & '
        tmpText = tmpText[:-3]
        order.append(tmpText)
    return(order)


def makeBoxPairs(O):
    return(list(itertools.combinations(O, 2)))


def renameAxes(axes, rD):
    try:
        N = len(axes)
    except:
        axes = [axes]
        N = 1
    for i in range(N):
        # set xticks
        xticksTextObject = axes[i].get_xticklabels()
        xticksList = [xticksTextObject[j].get_text() for j in range(len(xticksTextObject))]
        test_hasXLabels = (len(''.join(xticksList)) > 0)
        if test_hasXLabels:
            newXticksList = [rD.get(k, k) for k in xticksList]
            axes[i].set_xticklabels(newXticksList)
        
        # set xlabel
        xlabel = axes[i].get_xlabel()
        newXlabel = rD.get(xlabel, xlabel)
        axes[i].set_xlabel(newXlabel)
        # set ylabel
        ylabel = axes[i].get_ylabel()
        newYlabel = rD.get(ylabel, ylabel)
        axes[i].set_ylabel(newYlabel)
        

def addStat(ax, data, box_pairs, param, cond, test = 'MannWhitney', percentHeight = 98):
    refHeight = np.percentile(data[param].values, percentHeight)
    currentHeight = refHeight
    scale = ax.get_yscale()
    xTicks = ax.get_xticklabels()
    dictXTicks = {xTicks[i].get_text() : xTicks[i].get_position()[0] for i in range(len(xTicks))}
    for bp in box_pairs:
        c1 = data[data[cond] == bp[0]][param].values
        c2 = data[data[cond] == bp[1]][param].values
        if test=='Mann-Whitney':
            statistic, pval = st.mannwhitneyu(c1,c2)
        elif test=='t-test':
            statistic, pval = st.ttest_ind(c1,c2)
        text = 'ns'
        if pval < 0.05 and pval > 0.01:
            text = '*'
        elif pval < 0.01 and pval > 0.001:
            text = '**'
        elif pval < 0.001 and pval < 0.001:
            text = '***'
        elif pval < 0.0001:
            text = '****'
        ax.plot([bp[0], bp[1]], [currentHeight, currentHeight], 'k-', lw = 1)
        XposText = (dictXTicks[bp[0]]+dictXTicks[bp[1]])/2
        if scale == 'log':
            power = 0.006 * (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight*(refHeight**power)
        else:
            factor = 0.025 * (text=='ns') + 0.000 * (text!='ns')
            YposText = currentHeight + factor*refHeight
        ax.text(XposText, YposText, text, ha = 'center')
#         if text=='ns':
#             ax.text(posText, currentHeight + 0.025*refHeight, text, ha = 'center')
#         else:
#             ax.text(posText, currentHeight, text, ha = 'center')
        if scale == 'log':
            currentHeight = currentHeight*(refHeight**0.05)
        else:
            currentHeight =  currentHeight + 0.15*refHeight
    ax.set_ylim([ax.get_ylim()[0], currentHeight])
    

In [68]:
# Test of the averaging per cell routine

# data = GlobalTable_meca
# CondCol='drug'
# Parameters=['SurroundingThickness','EChadwick']
# Filters = [(GlobalTable_meca['Validated'] == 1)]
# AvgPerCell=True
# cellID='CellName'

# data_filtered = data
# for fltr in Filters:
#     data_filtered = data_filtered.loc[fltr]

# group = data_filtered.groupby(cellID)
# dictAggMean = getDictAggMean(data_filtered)
# data_filtered = group.agg(dictAggMean.pop(cellID)) #.reset_index(level=0, inplace=True)
# data_filtered.reset_index(level=0, inplace=True)
# data_filtered=data_filtered[[cellID]+[CondCol]+Parameters]
# print(data_filtered)

In [ ]:
# Test of a routine to remove points of a list of XY positions where at least 1 of the coordinates is 'nan'

# XYraw = np.array([[np.nan, 2, 3, np.nan, 5], [10,20,30,40,50]])
# XYraw = XYraw.T
# XY = XYraw[~np.isnan(XYraw).any(axis=1), :]
# X, Y = XY[:,0], XY[:,1]
# X, Y

In [ ]:
# Test of a routine to double each element in a list ; example [1, 2, 3] -> [1, 1, 2, 2, 3, 3]

# newnew_color_list = np.array([new_color_list, new_color_list])
# custom_color_list = list(np.array([new_color_list, new_color_list]).flatten(order='F'))
# custom_color_list

In [ ]:
# Test of makeOrder function

# print(makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs']))
# print(makeOrder(['A','B']))
# print(makeOrder(['A','B'], ['C','D']))
# print(makeOrder(['A','B'], ['C','D'], ['E','F']))

In [ ]:
# Test of makeBoxPairs function

# O = makeOrder(['none','doxycyclin'],['BSA coated glass','20um fibronectin discs'])
# makeBoxPairs(O)

### Exploratory Graphs

#### Classic

In [ ]:
rawMecaTable = getGlobalTable_meca()
# rawMecaTable.head()

In [ ]:
# rawMecaTable.loc[]
# Agréger tous les M450 WT sous un même nom

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M270') | (rawMecaTable['ExpType'] == 'DictyDB_M450')), (rawMecaTable['TpsComp'] == '1s')]
# Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M450')), (rawMecaTable['TpsComp'] == '1s')]
fig, ax = D1Plot(rawMecaTable, CondCol=['ExpType'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('M450 vs M270 for tps comp = 1s')
fig.show()
# rawMecaTable[Filters[0] & Filters[1] & Filters[2]]

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M450'))] #  | (rawMecaTable['ExpType'] == 'DictyDB_M450-Multi')
fig, ax00 = D1Plot(rawMecaTable, CondCol=['TpsComp'],Parameters=['SurroundingThickness','EChadwick'],\
                   Filters=Filters,AvgPerCell=False,cellID='CellName',stats=False,figSizeFactor = 1.8,markerSizeFactor=0.5)
fig.suptitle('M450, various rates')
fig.show()

In [ ]:
Filters = [(rawMecaTable['Validated'] == 1), ((rawMecaTable['ExpType'] == 'DictyDB_M270'))]
fig, ax01 = D1Plot(rawMecaTable, CondCol=['TpsComp'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
                   AvgPerCell=False,cellID='CellName',stats=False,figSizeFactor = 1.8,markerSizeFactor=0.5)
fig.suptitle('M270, various rates')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['medianThickness'] <= 1000)]
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['drug','substrate'],Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
fig.suptitle('3T3aSFL on patterns: Ct Field')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['none','doxycyclin'],['BSA coated glass','20um fibronectin discs']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName', co_order=co_order)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['drug','substrate'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
fig.suptitle('3T3aSFL on patterns: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder([['aSFL','aSFL-6FP'],['none','doxycyclin']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder([['aSFL','aSFL-6FP'],['none','doxycyclin']])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order)
fig.suptitle('3T3aSFL SHORT vs LONG linker: Compressions')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'], Filters=Filters, modelFit=True)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
# Same as above without the 2 thickest cells
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL'), (GlobalTable_ctField['medianThickness'] <= 700)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'], Filters=Filters, modelFit=True)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug_wo2LastPoints', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h)')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

In [ ]:
# xticksTextObject = ax.get_xticklabels()
# xticksList = [xticksTextObject[j].get_text() for j in range(len(xticksTextObject))]
# newXticksList = [renameDict1.get(k, k) for k in xticksList]
# newXticksList

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'], ['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=False,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: All Compressions')
fig.show()

In [163]:
figSubDir = 'ExploratoryPlots'
GlobalTable_meca_Py['compStartTimeThisDay_min'] = GlobalTable_meca_Py['compStartTimeThisDay']/60
GlobalTable_meca_Py

,date_x,cellName,cellID,manipID,compNum,compDuration,compStartTime,compAbsStartTime,compStartTimeThisDay,initialThickness,...,bead type,bead diameter,normal field,ramp field,compression duration,with fluo images,bacteria,comments_y,meanFluoPeakAmplitude,compStartTimeThisDay_min
0,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,1,1s,5.036,40650.429,0.000,79.623685,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,NaN,0.000000
1,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,2,1s,20.249,40665.642,15.213,56.730708,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,NaN,0.253550
2,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,3,1s,34.662,40680.055,29.626,46.999436,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,NaN,0.493767
3,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,4,1s,49.076,40694.469,44.040,88.049320,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,NaN,0.734000
4,20-08-04,M1_P1_C5,20-08-04_M1_P1_C5,20-08-04_M1,5,1s,63.487,40708.880,58.451,54.203678,...,M450,4503,10,['3'. '40'],1s,False,<NA>,<NA>,NaN,0.974183
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2084,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,6,1s,63.627,22157.136,7591.364,180.848673,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709,126.522733
2085,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,7,1s,75.543,22169.052,7603.280,234.485517,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709,126.721333
2086,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,8,1s,87.473,22180.982,7615.210,216.424678,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709,126.920167
2087,21-06-17,M2_P1_C9,21-06-17_M2_P1_C9,21-06-17_M2,9,1s,99.377,22192.886,7627.114,197.889093,...,M450,4503,5,['3'. '40'],1s,True,<NA>,<NA>,77.71709,127.118567


In [164]:
figSubDir = 'ExploratoryPlots'
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=False)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [165]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [170]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL-A8 - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-A8-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [171]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-A8'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL-A8 - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-A8-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [168]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['manipID','drug'],Parameters=['surroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL-6FP - Compressions - Differences with manip')
for i in range(len(ax)):
    xTicks = ax[i].get_xticklabels()
    for j in range(len(xTicks)):
        xTicks[j].set_fontsize(9)
archiveFig(fig, ax, name='3T3aSFL-6FP-Compressions-DifferencesWithManip_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [169]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
fig, ax = D2Plot(GlobalTable_meca_Py, XCol='compStartTimeThisDay_min',YCol='EChadwick', CondCol = ['date_x','drug'],\
           Filters=Filters, cellID = 'cellID', AvgPerCell=False, modelFit=False)
renameAxes(ax,renameDict1)
ax.legend(loc='upper right')
ax.set_xlabel('Time since start of experiment (min)')
fig.suptitle('3T3aSFL-6FP - Compressions - Evolution with time')
archiveFig(fig, ax, name='3T3aSFL-6FP-Compressions-EvolWithTime_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

#### Interactive

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]

p = D1PlotInteractive(GlobalTable_ctField, CondCol='drug',Parameters=['medianThickness','fluctuAmpli'],Filters=Filters)
p.children[0][0].title.text = '3T3aSFL on patterns: Ct Field'
p.children[0][0].title.text_font_size = '16pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D1PlotInteractive(GlobalTable_meca, CondCol='drug',Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,AvgPerCell=True,cellID='CellName')
p.children[0][0].title.text = '3T3aSFL on patterns: Compressions'
p.children[0][0].title.text_font_size = '14pt'
p.children[1][0].title.text = ''
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = 'drug', Filters=Filters)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = 'drug', Filters=Filters, cellID = 'CellName')
show(p)

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
p = D2PlotInteractive(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = 'drug', Filters=Filters, cellID = 'cellID',AvgPerCell=True)
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True)]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = '3T3aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL expressing linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: E(fluo)'
p.title.text_font_size = '16pt'
show(p)

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
p = D2PlotInteractive(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = 'drug', Filters=Filters, cellID = 'CellName',AvgPerCell=True)
p.title.text = 'aSFL-6FP expressing long linker: H(fluo)'
p.title.text_font_size = '16pt'
show(p)

### Clean Graphs

In [56]:
renameDict1 = {'SurroundingThickness':'Surrounding Thickness (nm)',\
               'surroundingThickness':'Surrounding Thickness (nm)',\
               'ctFieldThickness':'Thickness at low force (nm)',\
               'EChadwick': 'E Chadwick (Pa)',\
               'medianThickness': 'Median Thickness (nm)',\
               'fluctuAmpli': 'Fluctuations Amplitude (nm)',\
               'meanFluoPeakAmplitude' : 'Fluo Intensity (a.u.)',\
               'none & BSA coated glass':'control & non adherent',\
               'doxycyclin & BSA coated glass':'iMC & non adherent',\
               'none & 20um fibronectin discs':'control & adherent on fibro',\
               'doxycyclin & 20um fibronectin discs':'iMC & adherent on fibro',\
               'BSA coated glass & none':'control & non adherent',\
               'BSA coated glass & doxycyclin':'iMC & non adherent',\
               '20um fibronectin discs & none':'control & adherent on fibro',\
               '20um fibronectin discs & doxycyclin':'iMC & adherent on fibro',\
               'aSFL & none':'aSFL control',\
               'aSFL & doxycyclin':'aSFL iMC',\
               'aSFL-6FP & none':'aSFL-6FP control',\
               'aSFL-6FP & doxycyclin':'aSFL-6FP long-iMC',\
               'aSFL-A8 & none':'aSFL-A8 control',\
               'aSFL-A8 & doxycyclin':'aSFL-A8 iMC'}

figSubDir = 'CleanPlots'

#### Thickness and module

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['SurroundingThickness'] > 0)]
co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
                 Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='CellName', co_order=co_order)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()


In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['cell subtype'] == 'aSFL')]
co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['substrate','drug'],\
                 Parameters=['surroundingThickness','EChadwick'],Filters=Filters,\
                 AvgPerCell=True, cellID='cellID', co_order=co_order)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on diverse substrates: Compressions')
archiveFig(fig, ax, name='3T3aSFL_substrate&drug_SurroundingThickness&EChadwick_NEWTABLE', figSubDir = figSubDir)
fig.show()


In [ ]:
# Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
# co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
# fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
#                  Parameters=['EChadwick'],Filters=Filters,\
#                  AvgPerCell=True, cellID='CellName', co_order=co_order,statMethod = 'Mann-Whitney', statVerbose = 2)
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# fig.show()

# # test value should be one of the following: t-test_ind, t-test_welch, t-test_paired, Mann-Whitney, Mann-Whitney-gt, Mann-Whitney-ls, Levene, Wilcoxon, Kruskal.

In [ ]:
# Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
# co_order = makeOrder(['BSA coated glass','20um fibronectin discs'],['none','doxycyclin'])
# fig, ax = D1Plot(GlobalTable_meca, CondCol=['substrate','drug'],\
#                  Parameters=['SurroundingThickness','EChadwick'],Filters=Filters,\
#                  AvgPerCell=True, cellID='CellName', co_order=co_order, stats=False)
# renameAxes(ax,renameDict1)
# fig.suptitle('3T3aSFL on diverse substrates: Compressions')
# fig.show()
# # fig.savefig(todayFigDir + '//' + 'compressionsBSAvsFibro_woStats.png')

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['medianThickness'] <= 1000)]
co_order = makeOrder(['20um fibronectin discs'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_ctField, CondCol=['substrate','drug'],Parameters=['medianThickness','fluctuAmpli'],\
                 Filters=Filters,stats=True,co_order=co_order,figSizeFactor=0.5)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on patterns: Constant Field')
archiveFig(fig, ax, name='3T3aSFL_drug_medianThickness', figSubDir = figSubDir)

fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['ctFieldThickness'] <= 1000)]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['ctFieldThickness','ctFieldFluctuAmpli'],\
                 Filters=Filters,AvgPerCell=True,stats=True,co_order=co_order,box_pairs=box_pairs,figSizeFactor=1)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL on patterns: H and DH from meca expe')
archiveFig(fig, ax, name='3T3aSFL_drug_medianThickness_fromMeca_PYTHONTABLE', figSubDir = figSubDir)

fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-6FP'],['none','doxycyclin'])
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL short vs long linker: Compressions')
archiveFig(fig, ax, name='3T3aSFL_likerType&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca, CondCol=['cell subtype','drug'],Parameters=['SurroundingThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='CellName',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: Compressions')
archiveFig(fig, ax, name='3T3aSFL_likerType&drug_SurroundingThickness&EChadwick', figSubDir = figSubDir)
fig.show()

In [ ]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True), (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs')]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'],['none','doxycyclin'])
box_pairs=[('aSFL & none', 'aSFL & doxycyclin'),
 ('aSFL-A8 & none', 'aSFL-A8 & doxycyclin'),
 ('aSFL-6FP & none', 'aSFL-6FP & doxycyclin'),
 ('aSFL & none', 'aSFL-A8 & none'),
 ('aSFL & none', 'aSFL-6FP & none')]
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype','drug'],Parameters=['ctFieldThickness','EChadwick'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,box_pairs=box_pairs,stats=True)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL - All linker types: Compressions')
archiveFig(fig, ax, name='3T3aSFL_likerType&drug_ctFThickness&EChadwick_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

In [90]:
Filters = [(GlobalTable_meca_Py['validatedFit'] == True), (GlobalTable_meca_Py['validatedThickness'] == True),\
           (GlobalTable_meca_Py['substrate'] == '20um fibronectin discs'), (GlobalTable_meca_Py['drug'] == 'doxycyclin'),\
           (pd.isna(GlobalTable_meca_Py['meanFluoPeakAmplitude']) != True)]
co_order = makeOrder(['aSFL','aSFL-A8','aSFL-6FP'])
fig, ax = D1Plot(GlobalTable_meca_Py, CondCol=['cell subtype'],Parameters=['meanFluoPeakAmplitude'],\
                 Filters=Filters,AvgPerCell=True,cellID='cellID',co_order=co_order,stats=True,figSizeFactor=1.25)
renameAxes(ax,renameDict1)
fig.suptitle('3T3aSFL different cell lines\nLinker expression quantif by fluo')
archiveFig(fig, ax, name='3T3aSFL_likerType&_fluoExp_PYTHONTABLE', figSubDir = figSubDir)
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-83.25950030689842, 2938.0690149594357]


In [92]:
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'],\
                 Filters=Filters, modelFit=True, figSizeFactor = 0.8)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fitting condition: none with model y=ax+b
Y = 0.72424 X + -8.023
p-value on the 'a' coefficient: 2.4087e-08


Fitting condition: doxycyclin with model y=ax+b
Y = 1.8259 X + -154.51
p-value on the 'a' coefficient: 2.9304e-09




In [93]:
# Same as above without the 2 thickest cells
Filters = [(GlobalTable_ctField['validated'] == True), (GlobalTable_ctField['cell subtype'] == 'aSFL'),\
           (GlobalTable_ctField['medianThickness'] <= 700)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='medianThickness',YCol='fluctuAmpli',CondCol = ['drug'],\
                 Filters=Filters, modelFit=True, figSizeFactor = 0.8)
fig.suptitle('3T3aSFL - Dh = f(H)')
archiveFig(fig, ax, name='aSFL_Dh(h)_drug_wo2LastPoints', figDir = todayFigDir, figSubDir='ThicknessPlots')
fig.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Fitting condition: none with model y=ax+b
Y = 0.57405 X + 25.221
p-value on the 'a' coefficient: 4.3807e-04


Fitting condition: doxycyclin with model y=ax+b
Y = 1.3657 X + -73.332
p-value on the 'a' coefficient: 7.3580e-03




In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='SurroundingThickness',YCol='EChadwick',CondCol = ['substrate','drug'],\
           Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=False, modelType='y=A*exp(kx)')
fig.suptitle('3T3aSFL: E(h)')
archiveFig(fig, ax, name='aSFL_E(h)_drug&substrate', figDir = todayFigDir, figSubDir='')
fig.show()

#### Correlation with fluorescence

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
archiveFig(fig, ax, name='aSFL_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL'), (GlobalTable_meca['EChadwick'] <= 80000)]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: E(fluo)')
archiveFig(fig, ax, name='aSFL_iMC_E(fluo)_woLastPoint', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_mecaBis['validatedFit'] == True), ((GlobalTable_mecaBis['cell subtype'] == 'aSFL') | (GlobalTable_mecaBis['cell subtype'] == 'aSFL-A8')), (GlobalTable_mecaBis['drug'] == 'doxycyclin')]
fig, ax = D2Plot(GlobalTable_mecaBis, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL & aSFL-A8 expressing linker: E(fluo)')
archiveFig(fig, ax, name='aSFL&A8_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_mecaBis['validatedFit'] == True), (GlobalTable_mecaBis['drug'] == 'doxycyclin'), \
           ((GlobalTable_mecaBis['cell subtype'] == 'aSFL') | (GlobalTable_mecaBis['cell subtype'] == 'aSFL-A8')), \
           (GlobalTable_mecaBis['meanFluoPeakAmplitude'] <= 1200), (GlobalTable_mecaBis['EChadwick'] >= 2000)]
fig, ax = D2Plot(GlobalTable_mecaBis, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['cell subtype'], \
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL & aSFL-A8 expressing linker: E(fluo)')
archiveFig(fig, ax, name='aSFL&A8_iMC_E(fluo)_fluo-1200_&_E+2000', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude',YCol='SurroundingThickness',CondCol = ['drug'], Filters=Filters, cellID = 'CellName',AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: H(fluo)')
archiveFig(fig, ax, name='aSFL_iMC_H(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_ctField['validated'] == True)]
fig, ax = D2Plot(GlobalTable_ctField, XCol='meanFluoPeakAmplitude',YCol='medianThickness',CondCol = ['drug'],\
                 Filters=Filters, cellID = 'cellID', AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL expressing linker: medianH(fluo)')
archiveFig(fig, ax, name='aSFL_iMC_medianH(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: E(fluo)')
archiveFig(fig, ax, name='aSFL-6FP_iMC_E(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
#Same as above without the lonely point
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP'), (GlobalTable_meca['meanFluoPeakAmplitude'] <= 1000)]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='EChadwick', CondCol = ['drug'], \
                 Filters=Filters, cellID = 'CellName', AvgPerCell=True, modelFit=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: E(fluo)')
archiveFig(fig, ax, name='aSFL-6FP_iMC_E(fluo)_woLastPoint', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

In [ ]:
Filters = [(GlobalTable_meca['Validated'] == True), (GlobalTable_meca['cell subtype'] == 'aSFL-6FP')]
fig, ax = D2Plot(GlobalTable_meca, XCol='meanFluoPeakAmplitude', YCol='SurroundingThickness', CondCol = ['drug'],\
                 Filters=Filters, cellID = 'CellName',AvgPerCell=True)
renameAxes(ax,renameDict1)
fig.suptitle('aSFL-6FP expressing long linker: H(fluo)')
archiveFig(fig, ax, name='aSFL-6FP_iMC_H(fluo)', figDir = todayFigDir, figSubDir='FluoPlots')
fig.show()

## Utility scripts

### Create a test table to test different fits

In [ ]:
# Script !




testDir = os.path.join(dataDir, 'TestDataSet')
testFileName = 'testFitCompression.txt'
testFilePath = os.path.join(testDir, testFileName)

list_mecaFiles = [f for f in os.listdir(timeSeriesDataDir) \
                  if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".csv") \
                  and ('R40' in f))] # Change to allow different formats in the future
expDf = getExperimentalConditions()
tableDictTest = {}

for f in list_mecaFiles:
    print(f)
    tS_DataFilePath = os.path.join(timeSeriesDataDir, f)
    tsDF = pd.read_csv(tS_DataFilePath, ';')
    
    split_f = f.split('_')
    tsDF.dx, tsDF.dy, tsDF.dz, tsDF.D2, tsDF.D3 = tsDF.dx*1000, tsDF.dy*1000, tsDF.dz*1000, tsDF.D2*1000, tsDF.D3*1000
    thisManipID = split_f[0] + '_' + split_f[1]
    expDf['manipID'] = expDf['date'] + '_' + expDf['manip']
    thisExpDf = expDf.loc[expDf['manipID'] == thisManipID]
    DIAMETER = thisExpDf.at[thisExpDf.index.values[0], 'bead diameter']
    thisCellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
    Ncomp = max(tsDF['idxCompression'])
    
    for i in range(1, Ncomp+1): #Ncomp+1):
        thisCompHiD = thisCellID + '__' + str(i) + '__h'
        thisCompFiD = thisCellID + '__' + str(i) + '__f'
        print(thisCompHiD)
        
        thisCompDf = tsDF.loc[tsDF['idxCompression'] == i, :]
        iStart = (findFirst(tsDF['idxCompression'], i))
        iStop = iStart + thisCompDf.shape[0]
        
        # Delimit the start of the increase of B (typically the moment when the field decrease from 5 to 3)
        # and the end of its decrease (typically when it goes back from 3 to 5)
        
        listB = thisCompDf.B.values
        offsetStart, offsetStop = 0, 0
        minB, maxB = min(listB), max(listB)
        thresholdB = (maxB-minB)/50
        k = 0
        
        while (listB[k] > minB+thresholdB) or (listB[-1-k] > minB+thresholdB):
            offsetStart += int(listB[k] > minB+thresholdB)
            k += 1
        
        jStart = offsetStart
        jMax = np.argmax(thisCompDf.B)
        
        hCompr = (thisCompDf.D3.values[jStart:jMax+1] - DIAMETER)
        fCompr = (thisCompDf.F.values[jStart:jMax+1])
        
        tableDictTest[thisCompHiD] = hCompr
        tableDictTest[thisCompFiD] = fCompr
        
saveFile = open(testFilePath, 'w')
for k in tableDictTest.keys():
    saveFile.write(k)
    for i in range(len(tableDictTest[k])):
        saveFile.write(';')
        saveFile.write(str(tableDictTest[k][i]))
    saveFile.write('\n')
saveFile.close()

### Create a test table to try statistical tests

In [ ]:
# Script !

# Create a test table to try statistical tests

testDir = os.path.join(dataDir, 'TestDataSet')
GlobalTable_meca = getGlobalTable_meca()
table_ExpConditions = getExperimentalConditions()
table_fluo = getFluoData()
GlobalTable_meca = pd.merge(GlobalTable_meca, table_ExpConditions, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
GlobalTable_meca = pd.merge(GlobalTable_meca, table_fluo, how="left", left_on='CellName', right_on='cellID'
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)
print('Merged table has ' + str(GlobalTable_meca.shape[0]) + ' lines and ' + str(GlobalTable_meca.shape[1]) + ' columns.')

In [ ]:
# Table 1

testFileName = 'testStats01_allComp.csv'
testFilePath = os.path.join(testDir, testFileName)

Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['BSA coated glass','20um fibronectin discs'],['none','doxycyclin']])
data = GlobalTable_meca
CondCol=['substrate','drug']
Parameters=['SurroundingThickness','EChadwick']
AvgPerCell=False
cellID='CellName'

data_filtered = data
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]

NCond = len(CondCol)    
if NCond == 1:
    CondCol = CondCol[0]
elif NCond > 1:
    newColName = ''
    for i in range(NCond):
        newColName += CondCol[i]
        newColName += ' & '
    newColName = newColName[:-3]
    data_filtered[newColName] = ''
    for i in range(NCond):
        data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
    data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
    CondCol = newColName
    
if AvgPerCell:
    group = data_filtered.groupby(cellID)
    dictAggMean = getDictAggMean(data_filtered)
    data_filtered = group.agg(dictAggMean)

data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)

df_output = data_filtered[[cellID, 'CompNum', newColName] + Parameters]

# df_output.to_csv(testFilePath)

df_output

In [ ]:
# Table 2

testFileName = 'testStats02_avgPerCell.csv'
testFilePath = os.path.join(testDir, testFileName)

Filters = [(GlobalTable_meca['Validated'] == 1), (GlobalTable_meca['cell subtype'] == 'aSFL')]
co_order = makeOrder([['BSA coated glass','20um fibronectin discs'],['none','doxycyclin']])
data = GlobalTable_meca
CondCol=['substrate','drug']
Parameters=['SurroundingThickness','EChadwick']
AvgPerCell=True
cellID='CellName'

data_filtered = data
for fltr in Filters:
    data_filtered = data_filtered.loc[fltr]

NCond = len(CondCol)    
if NCond == 1:
    CondCol = CondCol[0]
elif NCond > 1:
    newColName = ''
    for i in range(NCond):
        newColName += CondCol[i]
        newColName += ' & '
    newColName = newColName[:-3]
    data_filtered[newColName] = ''
    for i in range(NCond):
        data_filtered[newColName] += data_filtered[CondCol[i]].astype(str)
        data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x + ' & ')
    data_filtered[newColName] = data_filtered[newColName].apply(lambda x : x[:-3])
    CondCol = newColName
    
if AvgPerCell:
    group = data_filtered.groupby(cellID)
    dictAggMean = getDictAggMean(data_filtered)
    data_filtered = group.agg(dictAggMean)

data_filtered.sort_values(CondCol, axis=0, ascending=True, inplace=True)

df_output = data_filtered[[cellID, 'CompNum', newColName] + Parameters]

# df_output.to_csv(testFilePath)

df_output

In [ ]:
# Table 3
# Fake data

testDir = os.path.join(dataDir, 'TestDataSet')

testFileName = 'testStats03_FakeData.csv'
testFilePath = os.path.join(testDir, testFileName)

Npop = 10
Npoints = 30
minAvg = 100
maxAvg = 600
step = (maxAvg - minAvg)/(Npop-1)
std = 250
dictFakeData = {}
np.random.seed(11)

for i in [1, 2, 3, 10]:
    dictFakeData['Distribution_' + str(i)] = np.random.normal(loc=minAvg + step*(i-1), scale=std, size=Npoints)

dfFakeData = pd.DataFrame(dictFakeData)

# dfFakeData.to_csv(testFilePath)

dfFakeData

In [ ]:
# Table 4
# Fake data 2

testDir = os.path.join(dataDir, 'TestDataSet')

testFileName = 'testStats04_FakeDataLarge.csv'
testFilePath = os.path.join(testDir, testFileName)

Npop = 10
Npoints = 300
minAvg = 100
maxAvg = 600
step = (maxAvg - minAvg)/(Npop-1)
std = 250
dictFakeData = {}
np.random.seed(11)

for i in [1, 2, 3, 10]:
    dictFakeData['Distribution_' + str(i)] = np.random.normal(loc=minAvg + step*(i-1), scale=std, size=Npoints)

dfFakeData = pd.DataFrame(dictFakeData)

# dfFakeData.to_csv(testFilePath)

dfFakeData

### Comparison of stat tests

#### With the fake data

In [ ]:
testDir = os.path.join(dataDir, 'TestDataSet')

# Table 3
# Fake data
testFileName = 'testStats03_FakeData.csv'
testFilePath = os.path.join(testDir, testFileName)

dfFakeData = pd.read_csv(testFilePath)
dfFakeData = dfFakeData.drop(columns=['Unnamed: 0'])

Ncol = len(dfFakeData.columns)

refCol = dfFakeData[dfFakeData.columns[0]]
boxPlotMatrix = []

for i in range(0,Ncol):
    boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
    if i > 0:
        print('Comparison between distribution 1 and ' + str(i+1))
        tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('tTest : ' + str(tTest.pvalue))
        wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('wilcox : ' + str(wilcox.pvalue))
        mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
        print('')

fig, ax = plt.subplots(1,1)
ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
fig.show()

In [ ]:
testDir = os.path.join(dataDir, 'TestDataSet')

# Table 4
# Fake data Large
testFileName = 'testStats04_FakeDataLarge.csv'
testFilePath = os.path.join(testDir, testFileName)

dfFakeData = pd.read_csv(testFilePath)
dfFakeData = dfFakeData.drop(columns=['Unnamed: 0'])

Ncol = len(dfFakeData.columns)

refCol = dfFakeData[dfFakeData.columns[0]]
boxPlotMatrix = []

for i in range(0,Ncol):
    boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
    if i > 0:
        print('Comparison between distribution 1 and ' + str(i+1))
        tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('tTest : ' + str(tTest.pvalue))
        wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('wilcox : ' + str(wilcox.pvalue))
        mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
        print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
        print('')

fig, ax = plt.subplots(1,1)
ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
fig.show()

In [ ]:
# Summary of results: numpy.random.seed = 11
    
dictResultsFakeData = {}
dictResultsFakeData['language'] = ['python'   , 'python', 'python'      , 'R'     , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsFakeData['test']     = ['ttest_ind', 'wilcox', 'mannwhitneyu', 't.test', 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsFakeData['1 vs 2']   = [0.2700     , 0.2711  , 0.1353        , 0.2701  , 0.2729       , 0.2700   , 0.2707   ]
dictResultsFakeData['1 vs 3']   = [0.0714     , 0.0822  , 0.0452        , 0.0715  , 0.0906       , 0.0714   , 0.0905   ]
dictResultsFakeData['1 vs 10']  = [1.33e-11   , 4.28e-06, 2.98e-09      , 1.44e-11, 5.799e-11    , 1.33e-11 , 5.96e-09 ]
dfResultFakeData = pd.DataFrame(dictResultsFakeData)
dfResultFakeData

In [ ]:
# Summary of results: numpy.random.seed = 11

dictResultsFakeDataLarge = {}
dictResultsFakeDataLarge['language'] = ['python'   , 'python', 'python'      , 'R'     , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsFakeDataLarge['test']     = ['ttest_ind', 'wilcox', 'mannwhitneyu', 't.test', 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsFakeDataLarge['1 vs 2']   = [0.0082     , 0.0049  , 0.0038        , 0.0082  , 0.0077       , 0.0082   , 0.0077   ]
dictResultsFakeDataLarge['1 vs 3']   = [1.26e-06   , 9.29e-06, 1.37e-06      , 1.27e-06, 2.75e-06     , 1.26e-06 , 2.74e-06 ]
dictResultsFakeDataLarge['1 vs 10']  = [1.74e-98   , 5.64e-48, 6.39e-74      , 2.2e-16 , 2.2e-16      , 1.74e-98 , 1.27e-73 ]
dictResultsFakeDataLarge = pd.DataFrame(dictResultsFakeDataLarge)
dictResultsFakeDataLarge

#### With real data

In [ ]:
# testDir = os.path.join(dataDir, 'TestDataSet')

# Table 1
# Avg Per Cell
testFileName = 'testStats01_AvgPerCell.csv'
testFilePath = os.path.join(testDir, testFileName)

dfAvgPerCell = pd.read_csv(testFilePath)
# dfAvgPerCell = dfAvgPerCell.drop(columns=['Unnamed: 0'])
dfAvgPerCell
categories = list(dfAvgPerCell['substrate & drug'].unique())
Ncat = len(categories)
for i in range(Ncat):
    for j in range(i,Ncat):
        if j != i:
            x = dfAvgPerCell.loc[dfAvgPerCell['substrate & drug'] == categories[i], 'EChadwick'].values
            y = dfAvgPerCell.loc[dfAvgPerCell['substrate & drug'] == categories[j], 'EChadwick'].values
            print('Comparison between ' + categories[i] + ' and ' + categories[j])
            tTest = st.ttest_ind(x, y)
            print('tTest : ' + str(tTest.pvalue))
            mannwhitneyu = st.mannwhitneyu(x, y)
            print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
            print('')

# refCol = dfFakeData[dfFakeData.columns[0]]
# boxPlotMatrix = []

# for i in range(0,Ncol):
#     boxPlotMatrix.append(dfFakeData[dfFakeData.columns[i]].values)
#     if i > 0:
#         print('Comparison between distribution 1 and ' + str(i+1))
#         tTest = st.ttest_ind(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('tTest : ' + str(tTest.pvalue))
#         wilcox = st.wilcoxon(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('wilcox : ' + str(wilcox.pvalue))
#         mannwhitneyu = st.mannwhitneyu(refCol.values, dfFakeData[dfFakeData.columns[i]].values)
#         print('mannwhitneyu : ' + str(mannwhitneyu.pvalue))
#         print('')

# fig, ax = plt.subplots(1,1)
# ax.boxplot(boxPlotMatrix,labels=dfFakeData.columns.values)
# ax.tick_params(axis='x', labelrotation = 15, labelsize = 7)
# fig.show()

In [ ]:
dictResultsAvgPerCell = {}
order = ['language','test','c & NA vs iMC & A','c & NA vs c & A','iMC & NA vs iMC & A','c & NA vs iMC & NA','iMC & NA vs c & A','c & A vs iMC & A']
dictResultsAvgPerCell['language']            = ['python_statAnot'   , 'python_statAnot' ,'python'   , 'python', 'R'      , 'R'          , 'Matlab' , 'Matlab' ]
dictResultsAvgPerCell['test']                = ['ttest_ind', 'mannwhitneyu', 'ttest_ind', 'mannwhitneyu', 't.test' , 'wilcox.test', 'ttest2' , 'ranksum']
dictResultsAvgPerCell['c & NA vs iMC & NA']  = [1.000e+00  , 5.669e-01     ,0.19226082553146542  , 0.047244130659518     , 0.1503   , 0.09502       , 0.1923       , 0.0945]
dictResultsAvgPerCell['iMC & NA vs c & A']   = [9.673e-02  , 8.625e-03     ,0.016121864893694285  ,0.0007187494204219925     , 0.04082  , 0.0009726    , 0.0161     , 0.0014]
dictResultsAvgPerCell['c & A vs iMC & A']    = [2.331e-02  , 1.376e-01     ,0.00388458593467288  , 0.01146586677766893     , 0.007326 , 0.02214      , 0.0039      , 0.0229]
dictResultsAvgPerCell['c & NA vs c & A']     = [1.000e+00  , 1.000e+00     ,0.6977550928576132  , 0.2884535746840493     , 0.6948   , 0.5838       , 0.6978     , 0.5769]
dictResultsAvgPerCell['iMC & NA vs iMC & A'] = [1.000e+00  , 1.000e+00     ,0.5573451346686198  , 0.41831870120029446,  0.5031  , 0.8387       , 0.5573      , 0.8366 ]
dictResultsAvgPerCell['c & NA vs iMC & A']   = [9.726e-01  , 1.000e+00     ,0.16209530366557973  , 0.14893352365754048     , 0.04353  , 0.3043       , 0.1621       , 0.2979 ]
dfResultsAvgPerCell = pd.DataFrame(dictResultsAvgPerCell)
dfResultsAvgPerCell[order]

# dfResultsAvgPerCell = dfResultsAvgPerCell.sort_values(by='test',ascending=False)
# dfResultsAvgPerCell

## Old Code

### Data formatting and filtering

TBC

In [ ]:
mecaDataFile = 'Global_MecaData.csv'
mecaDataFilePath = os.path.join(dataDir, mecaDataFile)
mecaDF = pd.read_csv(mecaDataFilePath, sep=';')
print('Extracted a table with ' + str(mecaDF.shape[0]) + ' lines and ' + str(mecaDF.shape[1]) + ' columns.')

mecaDF = mecaDF.rename(columns={"CellID": "CellName", "CellName": "CellID"})

In [ ]:
# mecaDF

In [ ]:
experimentalDataFile = 'ExperimentalConditions.csv'
experimentalDataFilePath = os.path.join(experimentalDataDir, experimentalDataFile)
expConditionsDF = pd.read_csv(experimentalDataFilePath, sep=';',header=0)
print('Extracted a table with ' + str(expConditionsDF.shape[0]) + ' lines and ' + str(expConditionsDF.shape[1]) + ' columns.')

# Cleaning the table
try:
    expConditionsDF = expConditionsDF.convert_dtypes()

    listTextColumns = []
    for col in expConditionsDF.columns:
        if expConditionsDF[col].dtype == 'string':
            listTextColumns.append(col)

    expConditionsDF[listTextColumns] = expConditionsDF[listTextColumns].apply(lambda x: x.str.replace(',','.'))

    expConditionsDF['scale pixel per um'] = expConditionsDF['scale pixel per um'].astype(float)
    expConditionsDF['optical index correction'] = \
              expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[0]).astype(float) \
            / expConditionsDF['optical index correction'].apply(lambda x: x.split('/')[1]).astype(float)
    expConditionsDF['magnetic field correction'] = expConditionsDF['magnetic field correction'].astype(float)
    expConditionsDF['with fluo images'] = expConditionsDF['with fluo images'].astype(bool)

    expConditionsDF['ramp field'] = \
    expConditionsDF['ramp field'].apply(lambda x: [x.split(';')[0], x.split(';')[1]] if not pd.isnull(x) else [])

except:
    print('Unexpected bug with the cleaning step')

In [ ]:
expConditionsDF

In [ ]:
# Unused for now
cellDescriptionDataFile = 'CellDescription.csv'
cellDescriptionDataFilePath = os.path.join(experimentalDataDir, cellDescriptionDataFile)
cellDescriptionDF = pd.read_csv(cellDescriptionDataFilePath, ',')
print('Extracted a table with ' + str(cellDescriptionDF.shape[0]) + ' lines and ' + str(cellDescriptionDF.shape[1]) + ' columns.')

In [ ]:
mecaDF['ManipID'] = mecaDF['ExpDay'] + '_' + mecaDF['CellName'].apply(lambda x: x.split('_')[0])
expConditionsDF['ManipID'] = expConditionsDF['date'] + '_' + expConditionsDF['manip']

mainMecaDF = pd.merge(
    expConditionsDF,
    mecaDF,
    how="inner",
    on='ManipID',
#     left_on=None,
#     right_on=None,
#     left_index=False,
#     right_index=False,
#     sort=True,
#     suffixes=("_x", "_y"),
#     copy=True,
#     indicator=False,
#     validate=None,
)

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
pd.reset_option('max_columns')

#### * Data filtering

In [ ]:
# pd.set_option('display.max_columns', None)
# mainMecaDF.head()

In [ ]:
# pd.reset_option('max_columns')
# mainMecaDF.columns

In [ ]:
mainMecaDF_f = mainMecaDF.loc[(mainMecaDF["Validated"] == 1)]
# mainMecaDF_f

In [ ]:
%matplotlib inline
listCells = mainMecaDF_f['CellID'].drop_duplicates().astype('string').values
timeSeriesDict = {}
for cell in listCells:
    currentCell_TimeSeriesData = getCellTimeSeriesData(cell)
    timeSeriesDict[cell] = currentCell_TimeSeriesData
start, stop = 80, 100
fig, axes = plt.subplots((stop-start),1, figsize = (7,4*(stop-start)))
fig.tight_layout()
for k in range(start, stop):
    if k < len(listCells):
        currentCell_TimeSeriesData = timeSeriesDict[listCells[k]]
        T = currentCell_TimeSeriesData['T'].values
        idxCompression = currentCell_TimeSeriesData['idxCompression'].values
        D3 = currentCell_TimeSeriesData['D3'].values
        maskConstant = (idxCompression == 0)
        maskCompression = (idxCompression > 0)
        axes[k - start].plot(T, D3*1000-4503, 'k-', linewidth = 0.5)
        axes[k - start].plot(T[maskCompression], D3[maskCompression]*1000-4503, 'ro', markersize=2)
        axes[k - start].plot(T[maskConstant], D3[maskConstant]*1000-4503, 'co', markersize=2)
        axes[k - start].set_title(listCells[k])
        axes[k - start].set_xlabel('T (s)')
        axes[k - start].set_ylabel('D3 (µm)')

In [ ]:
addExcludedCell('21-01-18_M1_P1_C2', 'passive')
addExcludedCell('21-01-18_M1_P1_C3', 'passive')
addExcludedCell('21-01-18_M1_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C4', 'passive')
addExcludedCell('21-01-21_M3_P1_C5', 'passive')
addExcludedCell('20-08-07_M1_P1_C6', 'too thick')
addExcludedCell('20-08-07_M1_P1_C62', 'too thick')

excludedCellsDict = getExcludedCells()
# # excludedMask = (mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())
# # mainMecaDF_f = mainMecaDF_f.loc[(mainMecaDF_f["CellID"].values not in excludedCellsDict.keys())]
# for i in range(len(excludedCellsDict)):
#     print('a')
# mainMecaDF_f["CellID"].drop_duplicates().astype('string').values
excludedCellsDict

In [ ]:
# currentCell_TimeSeriesData

In [ ]:
mainMecaDF_GroupedPerCell = mainMecaDF_f.groupby('CellID')
mainMecaDF_DataPerCell = mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "SurroundingThickness": np.median, "H0Chadwick" : np.median})
# mainMecaDF_GroupedPerCell.agg({"EChadwick": np.median, "D": lambda x: np.std(x, ddof=1)})
cols = ['date', 'manip', 'experimentType', 'drug', 'substrate',
       'objective magnification', 'scale pixel per um', 'objective immersion',
       'optical index correction', 'magnetic field correction', 'cell type',
       'cell subtype', 'bead type', 'bead diameter', 'normal field',
       'ramp field', 'compression duration', 'with fluo images', 'comments',
       'ManipID', 'ExpType', 'CellName', 'CellID']
mainMecaDF_DataPerCell.dropna(inplace = True)
mainMecaDF_DataPerCell = pd.merge(mainMecaDF_DataPerCell,
                                  mainMecaDF_f[cols].drop_duplicates(subset=['CellID']),
                                  how="inner",
                                  on='CellID',
                                  #     left_on='CellID',
                                  #     right_on='CellID',
                                  #     left_index=False,
                                  #     right_index=False,
                                  #     sort=True,
                                  #     suffixes=("_x", "_y"),
                                  #     copy=True,
                                  #     indicator=False,
                                  #     validate=None,
                                  )
# mainMecaDF_DataPerCell

In [ ]:
GlobalTable_meca_Count = GlobalTable_meca.groupby(['cell type', 'cell subtype', 'bead type', 'drug', 'substrate']).count()
GlobalTable_meca_Count.loc[:, ['CellID']].rename(columns={'CellID' : 'Count'})

### Other old code

In [ ]:
# fig, axes = plt.subplots(1,2, figsize = (8, 5))
# axes[0].plot(np.ones(len(resDict['nodrug']['median'])), resDict['nodrug']['median'], 'co')
# axes[0].plot(2*np.ones(len(resDict['doxy']['median'])), resDict['doxy']['median'], 'ro')
# axes[0].set_xlim(0.5, 2.5)
# axes[0].set_ylabel('Median Thickness (nm)')
# axes[0].set_xticks([1,2])
# axes[0].set_xticklabels(['Control','Doxycylin'])
# axes[1].plot(np.ones(len(resDict['nodrug']['fluctu'])), resDict['nodrug']['fluctu'], 'co')
# axes[1].plot(2*np.ones(len(resDict['doxy']['fluctu'])), resDict['doxy']['fluctu'], 'ro')
# axes[1].set_xlim(0.5, 2.5)
# axes[1].set_ylabel('Thickness Fluctuations (nm)')
# axes[1].set_xticks([1,2])
# axes[1].set_xticklabels(['Control','Doxycylin'])
# fig.savefig("C://Users//JosephVermeil//Desktop//ActinCortexAnalysis//DataAnalysis//constantField.png")
# fig.show()

In [ ]:
# Old code
conditions = ['nodrug', 'doxy']
correspondance = {conditions[0] : 'M1', conditions[1] : 'M2'}
allTimeSeriesDataFiles = [f for f in os.listdir(timeSeriesDataDir) if (os.path.isfile(os.path.join(timeSeriesDataDir, f)) and f.endswith(".txt"))]
dates = ['21-02-10']
resDict = {conditions[0] : {}, conditions[1] : {}}
for C in conditions:
    resDict[C]['accepted'] = []
    resDict[C]['rejected'] = []
    resDict[C]['median'] = []
    resDict[C]['fluctu'] = []
    for D in dates:
        for f in allTimeSeriesDataFiles:
            if correspondance[C] in f and D in f:
                split_f = f.split('_')
                cellID = split_f[0] + '_' + split_f[1] + '_' + split_f[2] + '_' + split_f[3]
                currentCellTS = getCellTimeSeriesData(cellID)
                D3 = currentCellTS.D3.values
                decile_1 = np.percentile(D3, 10)
                median = np.median(D3)
                decile_9 = np.percentile(D3, 90)
                if decile_1 < 0:
                    resDict[C]['rejected'].append(cellID)
                else:
                    resDict[C]['accepted'].append(cellID)
                    resDict[C]['median'].append(median)
                    resDict[C]['fluctu'].append(decile_9-decile_1)
#resDict

In [ ]:
GlobalTable_ctField = getGlobalTable_ctField()
table_ExpConditions = getExperimentalConditions()
GlobalTable_ctField = pd.merge(table_ExpConditions, GlobalTable_ctField, how="inner", on='manipID',
#     left_on=None,right_on=None,left_index=False,right_index=False,sort=True,
#     suffixes=("_x", "_y"),copy=True,indicator=False,validate=None,
)

# data=pd.DataFrame(dict(
#             x=[1, 2, 3, 4, 5],
#             y=[2, 5, 8, 2, 7],
#             desc=['A', 'A', 'C', 'd', 'E'],
#         ))

data = GlobalTable_ctField[['medianThickness','fluctuAmpli','cellID']]

source = ColumnDataSource(
        data=data
    )

# hover = HoverTool(
#         tooltips=[
#             ("index", "$index"),
#             ("(x,y)", "($x, $y)"),
#             ("desc", "@desc"),
#         ]
#     )

hover = HoverTool(
        tooltips=[
            ("medianThickness", "@medianThickness"),
            ("fluctuAmpli", "@fluctuAmpli"),
        ]
    )

p = figure(plot_width=300, plot_height=300, tools=[hover], title="Mouse over the dots")

p.circle('medianThickness', 'fluctuAmpli', size=20, source=data)

show(p)

data

In [ ]:
data=pd.DataFrame(dict(
            x=[1, 2, 3, 4, 5],
            y=[2, 5, 8, 2, 7],
            desc=['A', 'A', 'C', 'd', 'E'],
        ))

Conditions = list(data['desc'].unique())
NCond = len(Conditions)
data['X'] = 0
for i in range(NCond):
    mask = data['desc'] == Conditions[i]
    data.loc[mask, ['X']] = i+1
data.index = data.x
data = data.drop(['x'], axis = 1)

data.reset_index(level=0, inplace=True)
data